# **📚 Overall Project Proposal**
## Data Integrity in Action: Pre-processing Young People’s Records for Smarter Insights

From Data Chaos to Data-Driven Insights- This portfolio will present three interconnected projects that together form a phased, progressive, integrated, end-to-end data pipeline: (1) establishing a reliable data foundation, (2) enabling visual storytelling for stakeholders, and (3) piloting predictive/prescriptive approaches to personalise future programming.

Across all phases, the portfolio will demonstrate a full pipeline from raw data to actionable insights as well as my ability to handle real-world data challenges, build effective analytics workflows, and translate insights into meaningful organisational value.

## 🎨 Setup

In [1]:
#Set up the environment
import pandas as pd #Load the dataset from data/SSphere.csv into a DataFrame- used to "tidy" up and manipulate our data

import numpy as np #To use later for stats analysis- used for matrix and numerical calculations; foundation of pandas

import itertools

from scipy import stats #contains stats functions and is used to visualise probability distributions

import matplotlib.pyplot as plt #used for visualisations
%matplotlib inline

import matplotlib as mpl
from matplotlib import font_manager
from matplotlib.axes._axes import _log as matplotlib_axes_logger # Suppress Matplotlib specific warnings except errors
matplotlib_axes_logger.setLevel('ERROR')
import warnings
warnings.filterwarnings('ignore') # Suppressing all other general warnings except errors

import seaborn as sns #more user-friendly library used for visualisations
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# ML
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# time-series (optional)
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# **1️⃣ Project 1:**
## Data Integrity in Action: Pre-processing Young People’s Records for Smarter Insights

Project 1 focuses on establishing a strong and trustworthy data foundation. This stage involves importing, cleaning, standardising, and aggregating multiple datasets that currently exist across separate Airtable bases and formats. The aim is to remove duplication, fix inconsistent structures, and build a single, accurate record of each young person’s engagement. This ensures that downstream analytics are based on high-quality, well-organised data, improving efficiency, compliance, and reporting accuracy across the charity.

## 🔍 1.0 Understanding the Attendance Data

Each table (t1–4) contains hundreds of records representing either a young person or a school. Each record is described by 140 variables that capture a wide range of details about the participant, their school, and the events they attended. These variables include information such as student/school identifiers, programme details, demographics, school characteristics, regions, free school meal (FSM) status, age, attendance patterns, and additional survey-style responses (e.g., parental background, pronouns, ethnic background, and prior mentoring experience).

Together, these variables allow us to explore who attended, from where, under what circumstances, and how engagement patterns vary across different groups and programme types.

In [2]:
# (1.1) Import Datasets: Load data from csv's & Assign to t1, t2, t3 and t4
t1 = pd.read_csv('Data/Table 1-t1 (2).csv') #Import Table 1 (Sphere Attendance Data)
t2 = pd.read_csv('Data/Table 1-t2 (2).csv') #Import Table 1 (School Attendance Data)
t3 = pd.read_csv('Data/Table 1-t3 (2).csv') #Import Table 1 (School Outreach Attendance Data)
t4 = pd.read_csv('Data/Table 1-t4 (2).csv') #Import Table 1 (Free teaching Resource Recipient Data)
#t0 = pd.read_csv('Data/Table 1-t0.csv') #Import Table 1 (ALL- Updated, no concat required)

#Print raw shape of each of the imported tables
for t in [t1, t2, t3, t4]:
    print("Raw Shape:", t.shape)
    #print(t.shape, t.columns) #Note: t.describe too lengthy

Raw Shape: (1239, 148)
Raw Shape: (213, 148)
Raw Shape: (12, 148)
Raw Shape: (372, 148)


In [3]:
#Optional Shape check- replace .Shape with .Head to display the first few rows of each - 'tuple' object is not callable
##print(t0.shape())
#print(t1.shape())
#print(t2.shape())
#print(t3.shape())
#print(t4.shape())

#Optional Preview of Imported Dataset(s)
##Show first 5 rows of Table 0 (All Attendance Data)
##t0.head()

#Show first 5 rows of Table 1 (Sphere Attendance Data)
#t1.head()

#Show first 5 rows of Table 1 (School Attendance Data)
#t2.head()

#Show first 5 rows of Table 1 (School Outreach Attendance Data)
#t3.head()

#Show first 5 rows of Table 1 (Free teaching Resource Recipient Data)
#t4.head()

#t5.tail() #Show last 5 rows of Table 2 (Field ID Data)

In [4]:
# (1.2) Get a numbered index list of all unique values in main table > (t1) 'field_name' column to decide on columns of interest for later slicing/ grouping
#for i, col in enumerate(t1.columns):
#    print(f"{i}: {col}")

for i, col in enumerate(t1.columns):
    print(f"{i}: {col} ({t1[col].dtype})")

0: Student Name/School (object)
1: Record_ID (object)
2: Programmes x Table 1 (object)
3: STEAM Seasons (from Programmes x Table 1) (object)
4: Target Demographic (from Programmes x Table 1) (object)
5: Eligible Age Group** (from Programmes x Table 1) (object)
6: Partner Company (object)
7: Event_Sub-Type x Programmes (from Programmes x Table 1) (object)
8: Start Date (from Programmes x Table 1) (object)
9: End Date (from Programmes x Table 1) (object)
10: Year (from Programmes x Table 1) (int64)
11: Year-Quarter-Month (from Programmes x Table 1) (object)
12: Type (object)
13: Quantity (int64)
14: Formula to add records in Table 2 (object)
15: Calculate (object)
16: Link to Table 2 (object)
17: Notes (float64)
18: Student Name (object)
19: Programmes x Preferences #1 (object)
20: School (float64)
21: URN (from School) (float64)
22: Record_ID (from School) (float64)
23: Programmes x Schools #2 (float64)
24: DfE Schools Sync #2 (float64)
25: FTR School (float64)
26: # Summer24 FTR Recipi

In [5]:
# (1.3a) Define the t1 column rename mapping using indices to make grouping easier later
t1_column_mapping = {
    t1.columns[1]: "Record_ID",
    t1.columns[2]: "Event",
    t1.columns[3]: "STEAM Season",
    t1.columns[6]: "Partner",
    t1.columns[7]: "Event Sub-Type",
    t1.columns[10]: "Year",
    t1.columns[11]: "Year-Quarter-Month",
    t1.columns[12]: "Type",
    t1.columns[13]: "Quantity",
    t1.columns[21]: "URN_1", #Schools - fldFrRV3nU6882cct
    t1.columns[38]: "Region", #Yorkshire and Vs And
    t1.columns[46]: "LSOA (%)",
    t1.columns[54]: "State-funded (%)",
    t1.columns[61]: "FSM (%)",
    t1.columns[70]: "Age Today",
    t1.columns[78]: "Stream", # Stream based on Age today or Age on day of event?
    t1.columns[80]: "Age on Day of",
    t1.columns[83]: "P/G Attend Uni",
    t1.columns[84]: "P/G Work in STEAM",
    t1.columns[85]: "Pronouns",
    t1.columns[86]: "Ethnic Background",
    t1.columns[87]: "Physical / Virtual",
    t1.columns[88]: "UK-Based",
    t1.columns[89]: "Ethnic Background (Grouped)",
    t1.columns[93]: "Total Students Engaged",
    t1.columns[113]: "How Did You Hear",
    t1.columns[114]: "Where Did You Hear",
    t1.columns[116]: "QQ",
    t1.columns[128]: "Non-UK based country",
    t1.columns[129]: "Record_ID (Onb)",
    t1.columns[132]: "URN_2", #DfE - fldltehANDI6t4JeF
    t1.columns[133]: "URN_3", #FTR - fldr6kbUQIHp7UGaw
    t1.columns[134]: "URN_4", #Schools Outreach - fld7wF95fl0ljXsq2
    t1.columns[135]: "Onboarded Date/Time",
    t1.columns[136]: "First Time Attendee? (Y/N)",
    t1.columns[137]: "Record_ID (Unique)",
    t1.columns[138]: "Sign-up Date/Time",
    t1.columns[140]: "STEAM Tool",
    t1.columns[143]: "Record_ID (Fb)",
    t1.columns[144]: "# Fb",
    t1.columns[145]: "Fb sub date",
    t1.columns[146]: "Fb event date",
    t1.columns[147]: "Date/Time (1st Sign-up)",
}

#Alternative NOT using indices= Define the column rename mapping to make grouping easier later
#column_mapping = {column_mapping = {"Student Name/School": "Person"}

#Alternative way to column map
#t1_column_mapping = dict(zip(t1.columns.take([0, 1]),
#                                 ["DELETE", "Record_ID"]))

# (1.3b) Apply the Main Table renaming to all four attendance tables
#t0 = t0.rename(columns=t0_column_mapping)
t1 = t1.rename(columns=t1_column_mapping)
t2 = t2.rename(columns=t1_column_mapping)
t3 = t3.rename(columns=t1_column_mapping)
t4 = t4.rename(columns=t1_column_mapping)

# (1.3c) Get the list of new t1 column names (values from your mapping)
renamed_t1_columns = list(t1_column_mapping.values())

print("Renaming:", t1_column_mapping)

Renaming: {'Record_ID': 'Record_ID', 'Programmes x Table 1': 'Event', 'STEAM Seasons (from Programmes x Table 1)': 'STEAM Season', 'Partner Company': 'Partner', 'Event_Sub-Type x Programmes (from Programmes x Table 1)': 'Event Sub-Type', 'Year (from Programmes x Table 1)': 'Year', 'Year-Quarter-Month (from Programmes x Table 1)': 'Year-Quarter-Month', 'Type': 'Type', 'Quantity': 'Quantity', 'URN (from School)': 'URN_1', 'Region': 'Region', 'LSOA (%)': 'LSOA (%)', 'State-funded (%)': 'State-funded (%)', 'FSM (%)': 'FSM (%)', 'Age Today (from Contacts x  Eligibility) (from Eligibility x Preferences) (from Student Name)': 'Age Today', 'Stream (from Contacts x Eligibility) (from Eligibility x Preferences) (from Student Name)': 'Stream', 'Age on Day of': 'Age on Day of', 'Did any of your Parents/Guardians attend University? (from Contacts x Eligibility) (from Eligibility x Preferences) (from Student Name)': 'P/G Attend Uni', 'Do any of your Parents/Guardians work in STEAM fields? (from Cont

In [6]:
# (1.4) Build a DataFrame showing t1 old name vs new name
t1_rename_comparison = pd.DataFrame({
    "t1 Original": list(t1_column_mapping.keys()),
    "t1 Renamed": list(t1_column_mapping.values())
})

#Show the table directly
t1_rename_comparison

from IPython.display import display, HTML

#Nicely formatted HTML version
display(HTML(f"""
    <div style="flex: 1;">
        <h3>Attendance Data (t1) Column Mapping</h3>
        {t1_rename_comparison.to_html(index=False)}
    </div>
"""))

t1 Original,t1 Renamed
Record_ID,Record_ID
Programmes x Table 1,Event
STEAM Seasons (from Programmes x Table 1),STEAM Season
Partner Company,Partner
Event_Sub-Type x Programmes (from Programmes x Table 1),Event Sub-Type
Year (from Programmes x Table 1),Year
Year-Quarter-Month (from Programmes x Table 1),Year-Quarter-Month
Type,Type
Quantity,Quantity
URN (from School),URN_1


In [7]:
# (1.5a) Use only the renamed columns from each dataframe
t1_renamed = t1[renamed_t1_columns]
t2_renamed = t2[renamed_t1_columns]
t3_renamed = t3[renamed_t1_columns]
t4_renamed = t4[renamed_t1_columns]
#t0_renamed = t0[renamed_t0_columns]

# (1.5b) Concatenate the 4x Renamed Attendance Tables into 1
#Att = pd.concat([t1_renamed, t2_renamed, t3_renamed, t4_renamed], axis=0)

# (1.5b) Concatenate the 4x Renamed Attendance Tables into 1
Att = pd.concat(
    [t1_renamed, t2_renamed, t3_renamed, t4_renamed],
    axis=0,
    ignore_index=True  # <--- ensures a clean, unique integer index
)

# (1.5c) #Validate transformation process- Check the before/after shape of the concatenated Dataframe
print("Rows before:", len(t1_renamed),"+", len(t2_renamed),"+", len(t3_renamed),"+", len(t4_renamed), "| Rows after:", len(Att))
print("New Shape:", Att.shape)

#Remove Index and use Record_ID as Primary field
#Att = Att.set_index('Record_ID')

#Optional Preview of the first 3 rows only
Att.head(3)

#Convert expanded list to dataframe????????
#Att = pd.DataFrame(Att_exp)

Rows before: 1239 + 213 + 12 + 372 | Rows after: 1836
New Shape: (1836, 43)


,Record_ID,Event,STEAM Season,Partner,Event Sub-Type,Year,Year-Quarter-Month,Type,Quantity,URN_1,...,Onboarded Date/Time,First Time Attendee? (Y/N),Record_ID (Unique),Sign-up Date/Time,STEAM Tool,Record_ID (Fb),# Fb,Fb sub date,Fb event date,Date/Time (1st Sign-up)
0,recUrZnORsIVbVFwr,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1.0,NaN,...,11/7/2025 9:35am,EXISTING/Repeat Attendee,recMWiC7pw1ZO0hfe,10/7/2025 4:26pm,NaN,recwapMx1jjXPUZlp,1,19/7/2025,19/7/2025,10/7/2025
1,recqe2XUQ0eSk0CTf,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1.0,NaN,...,12/5/2025 2:37pm,NEW/First-time Attendee,recrPMUI4ZCTOkgop,30/4/2025 9:01pm,NaN,NaN,0,NaN,NaN,30/4/2025
2,recUdaEr0T20VEL5N,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1.0,NaN,...,18/6/2025 10:39am,EXISTING/Repeat Attendee,rec8fz0wx3s9gxbuH,18/6/2025 9:29am,NaN,NaN,0,NaN,NaN,26/6/2025


In [8]:
# (1.6) Make a copy of the newly concatenated df for easy referencing to the original (WITHOUT ANY columns that should remain anonymous e.g. AFTER we dropped the Person column)
#Att_anon = Att.drop(columns=["DELETE"])
Att_anon = Att.copy() #Use if I have no columns to drop

#Check the new shape of the concatenated and anonymised Dataframe
#print(Att_anon.shape)

#Preview the concatenated and anonymous dataframe
#Att_anon.head(3)

In [9]:
# (1.7) Apply DataType Transformations to Anonymous Dataset
#Change Quantity to Integer- this will be 'key' (matching column) for merge
Att_anon.loc[:, 'Quantity'] = pd.to_numeric(Att_anon['Quantity'], errors='coerce').astype('Int64')

#Change 'Quantity' column to Integer (Alternative to the .loc option?)
#Att_anon['Quantity'] = pd.to_numeric(Att_anon['Quantity'], errors='coerce').astype('Int64')

#Other columns to convert to integers
Att_cols_to_convert = ['URN_1', 'URN_2','URN_3', 'URN_4']
Att_anon[Att_cols_to_convert] = Att_anon[Att_cols_to_convert].astype('Int64') 

#Make a copy after applying transformations to concatenated dataset, before creating any slices of the original data
Att_tf = Att_anon.copy()

#Check the new shape of the transformed Dataframe
print(Att_tf.shape)

#Preview the result
Att_tf.head(3)

(1836, 43)


,Record_ID,Event,STEAM Season,Partner,Event Sub-Type,Year,Year-Quarter-Month,Type,Quantity,URN_1,...,Onboarded Date/Time,First Time Attendee? (Y/N),Record_ID (Unique),Sign-up Date/Time,STEAM Tool,Record_ID (Fb),# Fb,Fb sub date,Fb event date,Date/Time (1st Sign-up)
0,recUrZnORsIVbVFwr,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,<NA>,...,11/7/2025 9:35am,EXISTING/Repeat Attendee,recMWiC7pw1ZO0hfe,10/7/2025 4:26pm,NaN,recwapMx1jjXPUZlp,1,19/7/2025,19/7/2025,10/7/2025
1,recqe2XUQ0eSk0CTf,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,<NA>,...,12/5/2025 2:37pm,NEW/First-time Attendee,recrPMUI4ZCTOkgop,30/4/2025 9:01pm,NaN,NaN,0,NaN,NaN,30/4/2025
2,recUdaEr0T20VEL5N,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,<NA>,...,18/6/2025 10:39am,EXISTING/Repeat Attendee,rec8fz0wx3s9gxbuH,18/6/2025 9:29am,NaN,NaN,0,NaN,NaN,26/6/2025


In [10]:
# (1.8) Add any new columns for calculations
#Make a copy before adding a new column to the transformed dataframe
Att_new = Att_tf.copy()

#Create a new column called URN that extracts the first non-null URN value from columns a, b, c, or d for each row
Att_new['URN'] = Att_new[['URN_1','URN_2','URN_3','URN_4']].bfill(axis=1).iloc[:, 0] #could probably drop URN 1-4 after but optional

#Check the new shape of the Dataframe
print(Att_new.shape)

#Preview the result
Att_new.head()

(1836, 44)


,Record_ID,Event,STEAM Season,Partner,Event Sub-Type,Year,Year-Quarter-Month,Type,Quantity,URN_1,...,First Time Attendee? (Y/N),Record_ID (Unique),Sign-up Date/Time,STEAM Tool,Record_ID (Fb),# Fb,Fb sub date,Fb event date,Date/Time (1st Sign-up),URN
0,recUrZnORsIVbVFwr,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,<NA>,...,EXISTING/Repeat Attendee,recMWiC7pw1ZO0hfe,10/7/2025 4:26pm,NaN,recwapMx1jjXPUZlp,1,19/7/2025,19/7/2025,10/7/2025,<NA>
1,recqe2XUQ0eSk0CTf,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,<NA>,...,NEW/First-time Attendee,recrPMUI4ZCTOkgop,30/4/2025 9:01pm,NaN,NaN,0,NaN,NaN,30/4/2025,<NA>
2,recUdaEr0T20VEL5N,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,<NA>,...,EXISTING/Repeat Attendee,rec8fz0wx3s9gxbuH,18/6/2025 9:29am,NaN,NaN,0,NaN,NaN,26/6/2025,<NA>
3,recap8t8pYlhK4O7D,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,<NA>,...,NEW/First-time Attendee,recsamxoNTI98650L,9/7/2025 12:08pm,NaN,recw20mMvMLX0hJp7,1,19/7/2025,19/7/2025,9/7/2025,<NA>
4,recnGwj1cxBF7ihZy,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,<NA>,...,NEW/First-time Attendee,recbcAvydYk5DrsEm,1/6/2025 1:39pm,NaN,rec09fGZ9A55CgoJ7,1,19/7/2025,19/7/2025,1/6/2025,<NA>


In [11]:
# (1.9a) Misc
#This will hold an expanded list of school records
#Expand school rows into student-level rows (with safeguards)
Att_expansion = []

#Filter to display only rows where type contains the string 'Schools'
school_rows = Att_new[Att_new["Type"].str.contains("Schools", case=False, na=False)]

#Coerce Quantity to integer-like; invalid/missing -> NaN
school_rows["Quantity"] = pd.to_numeric(school_rows["Quantity"], errors="coerce")

#Loop through filtered rows
for _, row in school_rows.iterrows():
    qty = row["Quantity"]
    if pd.isna(qty) or int(qty) <= 0:
        continue  # skip missing or non-positive quantities/ Skip this row if quantity is missing

    base_id = str(row["Record_ID"])
    for i in range(1, int(qty) + 1):
        #Copy the full row and update Record_ID
        expanded_row = row.copy()
        expanded_row["Record_ID"] = f"{base_id}{i}"
        Att_expansion.append(expanded_row)
        
#Convert df_expansion list to dataframe
Att_expansion_df = pd.DataFrame(Att_expansion)

#Preview
print(Att_expansion_df.shape)

#Preview the expanded table with all columns
Att_expansion_df.head()

(10186, 44)


,Record_ID,Event,STEAM Season,Partner,Event Sub-Type,Year,Year-Quarter-Month,Type,Quantity,URN_1,...,First Time Attendee? (Y/N),Record_ID (Unique),Sign-up Date/Time,STEAM Tool,Record_ID (Fb),# Fb,Fb sub date,Fb event date,Date/Time (1st Sign-up),URN
1239,reccixyFsDiH2l9nk1,Adventures Season STEAM Goals 24,Adventures,Bank of America,STEM Goals,2024,2024 - Q4 - [11] November,Schools > Hannah,11,136717,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,136717
1239,reccixyFsDiH2l9nk2,Adventures Season STEAM Goals 24,Adventures,Bank of America,STEM Goals,2024,2024 - Q4 - [11] November,Schools > Hannah,11,136717,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,136717
1239,reccixyFsDiH2l9nk3,Adventures Season STEAM Goals 24,Adventures,Bank of America,STEM Goals,2024,2024 - Q4 - [11] November,Schools > Hannah,11,136717,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,136717
1239,reccixyFsDiH2l9nk4,Adventures Season STEAM Goals 24,Adventures,Bank of America,STEM Goals,2024,2024 - Q4 - [11] November,Schools > Hannah,11,136717,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,136717
1239,reccixyFsDiH2l9nk5,Adventures Season STEAM Goals 24,Adventures,Bank of America,STEM Goals,2024,2024 - Q4 - [11] November,Schools > Hannah,11,136717,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,136717


In [12]:
#(1.9b) Replace base school rows with expanded rows (concat)
Att_exp = pd.concat(
    [Att_new[~Att_new["Type"].str.contains("Schools", case=False, na=False)],
     Att_expansion_df],
    axis=0,
    ignore_index=True
)

#Filter out rows where Type contains 'Schools'
#Att_non_schools = Att_new[~Att_new["Type"].str.contains("Schools", case=False, na=False)]

#Concatenate the non-school rows with the expanded school rows to create an expanded DataFrame with all original columns
#Att_exp = pd.concat([Att_non_schools, Att_expansion_df], axis=0, ignore_index=True)

#Validate transformation process
assert Att_exp["Record_ID"].is_unique, "Duplicate Record_IDs found after expansion." #Check what I'm doing here?
print("Rows before:", len(Att_new), "| Rows after:", len(Att_exp))
print("New Shape:", Att_exp.shape)

#Preview the expanded table with all columns
Att_exp.head()

Rows before: 1836 | Rows after: 11425
New Shape: (11425, 44)


,Record_ID,Event,STEAM Season,Partner,Event Sub-Type,Year,Year-Quarter-Month,Type,Quantity,URN_1,...,First Time Attendee? (Y/N),Record_ID (Unique),Sign-up Date/Time,STEAM Tool,Record_ID (Fb),# Fb,Fb sub date,Fb event date,Date/Time (1st Sign-up),URN
0,recUrZnORsIVbVFwr,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,...,EXISTING/Repeat Attendee,recMWiC7pw1ZO0hfe,10/7/2025 4:26pm,NaN,recwapMx1jjXPUZlp,1,19/7/2025,19/7/2025,10/7/2025,<NA>
1,recqe2XUQ0eSk0CTf,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,...,NEW/First-time Attendee,recrPMUI4ZCTOkgop,30/4/2025 9:01pm,NaN,NaN,0,NaN,NaN,30/4/2025,<NA>
2,recUdaEr0T20VEL5N,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,...,EXISTING/Repeat Attendee,rec8fz0wx3s9gxbuH,18/6/2025 9:29am,NaN,NaN,0,NaN,NaN,26/6/2025,<NA>
3,recap8t8pYlhK4O7D,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,...,NEW/First-time Attendee,recsamxoNTI98650L,9/7/2025 12:08pm,NaN,recw20mMvMLX0hJp7,1,19/7/2025,19/7/2025,9/7/2025,<NA>
4,recnGwj1cxBF7ihZy,"""Achieve, Connect, Grow: Adventures Season Pan...",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,...,NEW/First-time Attendee,recbcAvydYk5DrsEm,1/6/2025 1:39pm,NaN,rec09fGZ9A55CgoJ7,1,19/7/2025,19/7/2025,1/6/2025,<NA>


In [13]:
# (1.10) Make a copy before making any modifications to the concatenated and merged data frame
Att_new1 = Att_exp.copy()

#Convert joined list to dataframe
Att_v1 = pd.DataFrame(Att_new1)
Att_v1.head()

#Check New Merged shape with x selected columns added from the DfE Index
print(Att_v1.shape)

#Generate Summary Stats for Numerical Data
##Descriptive Stats for different time-periods, streams, regions, types... maybe boss- repeat this process for sign-ups
Att_v1.describe()

(11425, 44)


,Year,Quantity,State-funded (%),Age Today,Age on Day of,Total Students Engaged,# Fb
count,11425.000000,11425.0,9957.000000,1237.000000,2432.000000,3006.000000,11425.000000
mean,2023.595536,127.137418,0.904991,15.413096,13.112664,18.536593,0.073961
std,0.605036,198.288198,0.293241,5.092979,3.841423,16.801112,0.330322
min,2023.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2023.000000,10.0,1.000000,12.000000,11.000000,10.000000,0.000000
50%,2024.000000,10.0,1.000000,16.000000,12.000000,10.000000,0.000000
75%,2024.000000,200.0,1.000000,18.000000,15.000000,28.000000,0.000000
max,2025.000000,650.0,1.000000,63.000000,63.000000,90.000000,5.000000


In [14]:
#Check how many values in the Record_ID column are Unique (no duplicates), NOT Unique (contain duplicates), and Nulls (contain missing values)
# --- Record_ID checks ---
#print("Total Att Records (unique):", Att_v1['Record_ID'].nunique())   #Print no. of unique entries
#print("Duplicate Total Att Records (max frequency):", Att_v1['Record_ID'].value_counts().max())  # Largest duplicate count
#print("Total duplicated Record_IDs:", Att_v1['Record_ID'].duplicated().sum())  # Total duplicated rows
#print("Null Record_ID count:", Att_v1['Record_ID'].isna().sum())      # Nulls

#print("\n" + "-"*50 + "\n") # Line break

# --- Record_ID (Unique) checks ---
#print("Unique Att Records (unique):", Att_v1['Record_ID (Unique)'].nunique())  #Print no. of unique entries
#print("Duplicate Unique Att Records (max frequency):", Att_v1['Record_ID (Unique)'].value_counts().max())  # Largest duplicate count
#print("Total duplicated Record_ID (Unique) rows:", Att_v1['Record_ID (Unique)'].duplicated().sum())  # Total duplicated rows
#print("Null Record_ID (Unique) count:", Att_v1['Record_ID (Unique)'].isna().sum())  # Nulls

In [15]:
# Define columns to summarise
v1_columns_to_check = ['Record_ID', 'Record_ID (Unique)']

v1_summary_list = []

for col in v1_columns_to_check:
    total_rows = len(Att_v1)
    null_count = Att_v1[col].isna().sum()
    non_null_data = Att_v1[col].dropna()
    unique_count = non_null_data.nunique()
    
    # Max times a single value is repeated
    if len(non_null_data) > 0:
        max_dup_freq = non_null_data.value_counts().max()
        total_duplicates = non_null_data.duplicated(keep='first').sum()
    else:
        max_dup_freq = 0
        total_duplicates = 0
    
    v1_summary_list.append({
        'Column': col,
        'Total_Rows': total_rows,
        'Unique_Count': unique_count,
        'Max_Duplicate_Frequency': max_dup_freq,
        'Total_Duplicated_Rows': total_duplicates,
        'Null_Count': null_count
    })

# Convert to DataFrame
v1_summary_df = pd.DataFrame(v1_summary_list)

v1_summary_df

,Column,Total_Rows,Unique_Count,Max_Duplicate_Frequency,Total_Duplicated_Rows,Null_Count
0,Record_ID,11425,11425,1,0,0
1,Record_ID (Unique),11425,929,14,308,10188


In [16]:
# Filter non-null Record_ID (Unique)
non_null_unique = Att_v1[Att_v1['Record_ID (Unique)'].notna()]

# Count duplicates (excluding first occurrence)
total_duplicates = non_null_unique.duplicated(subset=['Record_ID (Unique)'], keep='first').sum()

print(f"Total rows = {len(Att_v1)}, Null Record_ID (Unique) rows = {Att_v1['Record_ID (Unique)'].isna().sum()} (Schools), so Non-null rows = {len(non_null_unique)} (Young People).")
print(f"Of those {len(non_null_unique)} non-null rows, {non_null_data.nunique()} are unique participants, {total_duplicates} are duplicates (the most repeated ID [frequent attendee] appears {non_null_unique['Record_ID (Unique)'].value_counts().max()} times).\n")
print("Record_ID is clean — mostly unique, no duplicates, no missing values.")
print("Record_ID (Unique) is messy — many rows are duplicates or missing, and the most repeated ID appears 12 times.")

Total rows = 11425, Null Record_ID (Unique) rows = 10188 (Schools), so Non-null rows = 1237 (Young People).
Of those 1237 non-null rows, 929 are unique participants, 308 are duplicates (the most repeated ID [frequent attendee] appears 14 times).

Record_ID is clean — mostly unique, no duplicates, no missing values.
Record_ID (Unique) is messy — many rows are duplicates or missing, and the most repeated ID appears 12 times.


In [17]:
# Make pandas display wider columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 200)

# Filter out nulls
non_null_att = Att_v1[Att_v1['Record_ID (Unique)'].notna()]

# Group by Record_ID (Unique), including singles
summary_all = (
    non_null_att.groupby('Record_ID (Unique)')
    .agg(
        Total_Count=('Record_ID', 'count'),
        Associated_Record_IDs=('Record_ID', lambda x: ", ".join(map(str, x)))
    )
    .reset_index()
)

# Calculate duplicates excluding first occurrence
summary_all['Duplicates_Excluding_First'] = summary_all['Total_Count'] - 1

# Sort by Duplicates_Excluding_First descending
summary_all_sorted = summary_all.sort_values(by='Duplicates_Excluding_First', ascending=False).reset_index(drop=True)

print("Summary view of all non-null Record_ID (Unique), including singles:")
summary_all_sorted.head(50)


Summary view of all non-null Record_ID (Unique), including singles:


,Record_ID (Unique),Total_Count,Associated_Record_IDs,Duplicates_Excluding_First
0,recCHtY6Lko1UKXDa,14,"recWj2cLSnFIQFgNn, rec54Q5W3dhAVGEvS, recpTVqiF1jUjMhys, recHvIB5M2yAf7hqj, recpEpECHwg7VjAF9, recjVYp7V85OKclaS, recK596rfwO33F8In, recQ3JJgiiryrGSpp, recinzrp7nvnUegcn, recJj2u9WYvgba3Cb, recPNIhs2bQCAbRfo, recVX7zJJrp2XThhM, recDNw0BfbLevkjtT, recM5eLdAokXH7EJu",13
1,recHXCgIG2d04ssNJ,12,"reckqSiOG2nYiCEAq, recB85Kt7vwkI2RzV, recNIT3rhtTcQ5LHT, recZd3Ro8V7qIeUL4, recM5BaFrGW8EyouM, reciermUItTbdF39v, rec5ciGotAEdhzhhA, recRm9pjIRiORg9k7, recc9NBRVUbzFCB1e, recrnGG5EoHRvjw7Y, recE9xBjRukft6lBn, recN8S0UZaa6JnvId",11
2,recH9QXD7vJ0CIi9r,9,"recdJCQoNmKyvLKcY, recYuXgyma7NAbkM1, recIfQFf6kGGEnpwd, recUDBmpqENDldJtG, rec6CCWRLuFrwpIH3, recPm6YYLY6vqnfSo, recSQhcWChWno7ju9, recpBKqUid2pFlAuJ, recELFi0FqtKVS0JI",8
3,recwbGGMf4KsuhAMQ,8,"recL3ogCPHyBDc5Wr, recE4pUW1LZkDfEv7, rec2tRCODnZeHnpWR, recdUCPmVmZ2jP9OQ, rec4V7Gu6FtrRTpKw, recIGB6VzVthavGm0, rec4Ih6Vp9INrbAEL, receTN4QzGMaQxsiN",7
4,recHpVO4pYFyYzr7l,7,"recKycGBFAI0Rv6Vb, rect64OLuiT7Z2XDV, recBPBwt55z8839yG, reczNMMMLKf3Fv8j4, recTUZhdTrfU1SnEm, reczpAX0qEhC5u0Qr, rec5VRR9kKvgusNlh",6
5,recAE7iIa6ivQ2Twc,7,"rec2AOohpMzyLHko0, recrODUUeV64DXnjY, recRf2Bow3YPqV6nV, recPwEideXSDb4NFE, rec05ilwMkvuYLK4e, recU89bbxhfxooybg, recaH1tXM2wW7qT0i",6
6,rec5jnEo1YpnkCNm1,6,"recRbLMAN5UNkbyTt, rec6BHU6L2KF0WkaB, recIogwofBcezvkZ6, reciFnqPdcvJOg2zm, rec3od6CGJgXJtz5o, recJqCts4M4p6ujBF",5
7,recTjTBqZLtc38SAk,6,"reciBvQyNiSgkljlX, rec6PAwgxYoLo04ss, reciGhXd6VbfAd9lE, rec2AV8oISaIgs9x0, recsnrcqXyM32EMYI, recLgnzBxaNR1efxQ",5
8,rec1tsvSA5WmSUg26,6,"recgyjLSRiJePOI9A, recwQXYLWcBqw0gzA, recdYVaxgu6kacIiW, recZNUsrzATq10hEI, recGxLBjulCnUVpgW, recg3jXwKeyXG1KAz",5
9,recCbb8CxHrUO8ldL,6,"recdgBSnokWWARSQA, recngZ9wcMVxVlBTN, recgcNjHEJeJQiOfs, recQUTfR5EOEXoBI5, recgTl66nStXZtR3u, reckdmOwCPb3Jfrag",5


## 🔍 2.0 Understanding the DfE data

This table contains hundreds of records, each representing a UK-based school. Every record is described by 139 variables capturing a wide range of details about the school’s identity, status, demographics, and performance.

These variables include: Identifiers (e.g. URN, establishment number, LA codes, and trust/federation links); School characteristics (e.g. establishment type, phase of education, age range, gender intake, religious character, admissions policy, and capacity); Demographics (e.g. total pupils, number of boys and girls, percentage of pupils eligible for Free School Meals (FSM), and special educational needs (SEN) provisions); Governance and oversight (e.g. Ofsted inspection dates, ratings, special measures, and inspectorate details) and Location data (e.g. address, postcode, region, parliamentary constituency, LSOA/MSOA codes, and geographic coordinates). Additional attributes included school website, leadership information, provision types, accreditations, and sponsor/federation details.

Together, these fields provide a comprehensive view of schools across England, enabling analyses on school type, size, region, governance, performance, and student demographics.

In [18]:
# (2.1) Import dataset
t6 = pd.read_csv('Data/Table 3-DfE-ID.csv', low_memory=False) #Error with upload of Opensource data file

#Print raw shape of the imported table
print("Raw Shape:", t6.shape)

Raw Shape: (25511, 140)


In [19]:
# (2.2) Get a numbered index list of all unique values in 'field_name' column to decide on columns of interest
for i, col in enumerate(t6.columns):
    print(f"{i}: {col}")
#].dtype})") ???

0: URN
1: LA (code)
2: LA (name)
3: EstablishmentNumber
4: EstablishmentName
5: TypeOfEstablishment (code)
6: TypeOfEstablishment (name)
7: EstablishmentTypeGroup (code)
8: EstablishmentTypeGroup (name)
9: EstablishmentStatus (code)
10: EstablishmentStatus (name)
11: ReasonEstablishmentOpened (code)
12: ReasonEstablishmentOpened (name)
13: OpenDate
14: ReasonEstablishmentClosed (code)
15: ReasonEstablishmentClosed (name)
16: CloseDate
17: PhaseOfEducation (code)
18: PhaseOfEducation (name)
19: StatutoryLowAge
20: StatutoryHighAge
21: Boarders (code)
22: Boarders (name)
23: NurseryProvision (name)
24: OfficialSixthForm (code)
25: OfficialSixthForm (name)
26: Gender (code)
27: Gender (name)
28: ReligiousCharacter (code)
29: ReligiousCharacter (name)
30: ReligiousEthos (name)
31: Diocese (code)
32: Diocese (name)
33: AdmissionsPolicy (code)
34: AdmissionsPolicy (name)
35: SchoolCapacity
36: SpecialClasses (code)
37: SpecialClasses (name)
38: CensusDate
39: NumberOfPupils
40: NumberOfBoys


In [20]:
# (2.3a) Define the column rename mapping using indices to make grouping easier later
t6_column_mapping = {
    t6.columns[0]: "URN",
    t6.columns[2]: "Local Authority Name",
    t6.columns[4]: "Establishment Name",
    t6.columns[6]: "Type of Establishment",
    t6.columns[8]: "Establishment Type Group",
    t6.columns[19]: "Statutory_Low_Age",
    t6.columns[20]: "Statutory_High_Age",
    t6.columns[39]: "No. of Pupils",
    t6.columns[42]: "Percentage FSM",
    t6.columns[59]: "Street",
    t6.columns[60]: "Locality",
    t6.columns[62]: "Town",
    t6.columns[64]: "Postcode",
    t6.columns[125]: "Ofsted Rating",
    t6.columns[137]: "LSOA (code)",
    t6.columns[138]: "FSM",
}

# (2.3b) Apply the renaming to DfE table
t6 = t6.rename(columns=t6_column_mapping)

# (2.3c) Get the list of new column names (values from your mapping)
renamed_t6_columns = list(t6_column_mapping.values())

print("Renaming:", t6_column_mapping)

Renaming: {'URN': 'URN', 'LA (name)': 'Local Authority Name', 'EstablishmentName': 'Establishment Name', 'TypeOfEstablishment (name)': 'Type of Establishment', 'EstablishmentTypeGroup (name)': 'Establishment Type Group', 'StatutoryLowAge': 'Statutory_Low_Age', 'StatutoryHighAge': 'Statutory_High_Age', 'NumberOfPupils': 'No. of Pupils', 'PercentageFSM': 'Percentage FSM', 'Street': 'Street', 'Locality': 'Locality', 'Town': 'Town', 'Postcode': 'Postcode', 'OfstedRating (name)': 'Ofsted Rating', 'LSOA (code)': 'LSOA (code)', 'FSM': 'FSM'}


In [21]:
# (2.4) Build a DataFrame showing t6 old name vs new name
t6_rename_comparison = pd.DataFrame({
    "T6 Original": list(t6_column_mapping.keys()),
    "T6 Renamed": list(t6_column_mapping.values())
})

#Show the table directly
t6_rename_comparison

from IPython.display import display, HTML

#Nicely formatted HTML version
display(HTML(f"""
    <div style="flex: 1;">
        <h3>DfE Schools Data (T6) Column Mapping</h3>
        {t6_rename_comparison.to_html(index=False)}
    </div>
"""))

T6 Original,T6 Renamed
URN,URN
LA (name),Local Authority Name
EstablishmentName,Establishment Name
TypeOfEstablishment (name),Type of Establishment
EstablishmentTypeGroup (name),Establishment Type Group
StatutoryLowAge,Statutory_Low_Age
StatutoryHighAge,Statutory_High_Age
NumberOfPupils,No. of Pupils
PercentageFSM,Percentage FSM
Street,Street


In [22]:
# (2.5a) Use only the renamed columns from the dataframe for concatenation 
DfE = t6[renamed_t6_columns]

# (2.5b) Concatenate the Renamed DfE Tables into 1 - #No need to concat
#DfE = pd.concat([t6_renamed], axis=0)

# (2.5c) #Validate transformation process- Check the before/after shape of the concatenated Dataframe - #No need to validate
#print("Rows before:", len(t6_renamed), "| Rows after:", len(DfE))
print(DfE.shape)

#Check that all the values in the Record_ID column are Unique (no duplicates) - #No need to check for duplicates
#print(DfE['Record_ID'].value_counts().max())

#Check that all the values in the Record_ID (Unique) column are not Unique (contain duplicates) - #No need to check for duplicate values
#print(DfE['Record_ID (Unique)'].value_counts().max())

#Remove Index and use Record_ID as Primary field
#DfE = DfE.set_index('Record_ID')

#Optional Preview of the first 3 rows only
DfE.head(3)

(25511, 16)


,URN,Local Authority Name,Establishment Name,Type of Establishment,Establishment Type Group,Statutory_Low_Age,Statutory_High_Age,No. of Pupils,Percentage FSM,Street,Locality,Town,Postcode,Ofsted Rating,LSOA (code),FSM
0,100000,City of London,The Aldgate School,Voluntary aided school,Local authority maintained schools,3.0,11.0,271.0,18.1,St James's Passage,Duke's Place,London,EC3A 5DE,Outstanding,E01032739,49.0
1,100001,City of London,City of London School for Girls,Other independent school,Independent schools,7.0,18.0,739.0,0.0,St Giles' Terrace,Barbican,London,EC2Y 8BB,NaN,E01000002,0.0
2,100002,City of London,St Paul's Cathedral School,Other independent school,Independent schools,4.0,13.0,269.0,0.0,2 New Change,NaN,London,EC4M 9AD,NaN,E01032739,0.0


In [23]:
# (2.6) #No need to drop or delete any columns
#Make a copy of the newly concatenated df for easy referencing to the original (WITHOUT ANY columns that should remain anonymous i.e. AFTER we dropped the _ column)
#DfE_anon = DfE.drop(columns=["DELETE"])
DfE_anon = DfE.copy() #Use if I have no columns to drop

#Check the new shape of the concatenated and anonymised Dataframe
#print(DfE_anon.shape)

#Preview the concatenated and anonymous dataframe
#DfE_anon.head(3)

In [24]:
# (2.7) Apply DataType Transformations to 'Anonymous' Dataset
#Ensure FSM values are Integers
DfE_anon.loc[:, 'FSM'] = pd.to_numeric(DfE_anon['FSM'], errors='coerce').astype('Int64')

#Change 'FSM' column to Integer (Alternative to the .loc option?)
#if "FSM" in DfE.columns:
#    DfE_anon.loc[:, col] = pd.to_numeric(DfE_anon[col], errors='coerce').astype('Int64')

#Other columns to convert to integers #for col in DfE_cols_to_convert:
DfE_cols_to_convert = ['Statutory_Low_Age', 'Statutory_High_Age', 'No. of Pupils']
DfE_anon[DfE_cols_to_convert] = DfE_anon[DfE_cols_to_convert].astype('Int64')

#Change 'FSM' column to Integer (Alternative to the .loc option?)
#DfE_anon.loc[:, 'Statutory_Low_Age', 'Statutory_High_Age', 'No. of Pupils', 'FSM'] = pd.to_numeric(DfE_anon['Statutory_Low_Age', 'Statutory_High_Age', 'No. of Pupils', 'FSM'], errors='coerce').astype('Int64')

#Make a copy after applying transformations to concatenated dataset, before creating any slices of the original data
DfE_tf = DfE_anon.copy()

#Check the new shape of the transformed Dataframe
print(DfE_tf.shape)

#Preview the result
DfE_tf.head(3)

(25511, 16)


,URN,Local Authority Name,Establishment Name,Type of Establishment,Establishment Type Group,Statutory_Low_Age,Statutory_High_Age,No. of Pupils,Percentage FSM,Street,Locality,Town,Postcode,Ofsted Rating,LSOA (code),FSM
0,100000,City of London,The Aldgate School,Voluntary aided school,Local authority maintained schools,3,11,271,18.1,St James's Passage,Duke's Place,London,EC3A 5DE,Outstanding,E01032739,49
1,100001,City of London,City of London School for Girls,Other independent school,Independent schools,7,18,739,0.0,St Giles' Terrace,Barbican,London,EC2Y 8BB,NaN,E01000002,0
2,100002,City of London,St Paul's Cathedral School,Other independent school,Independent schools,4,13,269,0.0,2 New Change,NaN,London,EC4M 9AD,NaN,E01032739,0


In [25]:
# (2.8) Add any new columns for calculations
#Make a copy before adding a new column to the transformed dataframe
DfE_new = DfE_tf.copy()

#Create a new column called x that does y for each row
#DfE_new['x'] = DfE_new[['y']].bfill(axis=1).iloc[:, 0]

#Check the new shape of the Dataframe
print(DfE_new.shape)

#Preview the result
DfE_new.head()

(25511, 16)


,URN,Local Authority Name,Establishment Name,Type of Establishment,Establishment Type Group,Statutory_Low_Age,Statutory_High_Age,No. of Pupils,Percentage FSM,Street,Locality,Town,Postcode,Ofsted Rating,LSOA (code),FSM
0,100000,City of London,The Aldgate School,Voluntary aided school,Local authority maintained schools,3,11,271,18.1,St James's Passage,Duke's Place,London,EC3A 5DE,Outstanding,E01032739,49
1,100001,City of London,City of London School for Girls,Other independent school,Independent schools,7,18,739,0.0,St Giles' Terrace,Barbican,London,EC2Y 8BB,NaN,E01000002,0
2,100002,City of London,St Paul's Cathedral School,Other independent school,Independent schools,4,13,269,0.0,2 New Change,NaN,London,EC4M 9AD,NaN,E01032739,0
3,100006,Camden,Heath School,Pupil referral unit,Local authority maintained schools,11,16,19,68.4,Agincourt Road,NaN,London,NW3 2NY,Good,E01000887,13
4,100007,Camden,Camden Primary Pupil Referral Unit,Pupil referral unit,Local authority maintained schools,5,11,19,100.0,"Robson House, Netley Campus",74 Stanhope Street,London,NW1 3EX,Good,E01035710,19


In [26]:
# (2.9) Misc
DfE_matched = DfE_new[DfE_new['URN'].notna() & DfE_new['URN'].isin(Att_v1['URN'].dropna())]

#Convert t6_matched list to dataframe
DfE_matched_df = pd.DataFrame(DfE_matched)
DfE_matched_df.head()

print(DfE_matched_df.shape) #Check number of schools from DfE_Index that appear in the consolidated attendance table

DfE_matched_df.head()

(337, 16)


,URN,Local Authority Name,Establishment Name,Type of Establishment,Establishment Type Group,Statutory_Low_Age,Statutory_High_Age,No. of Pupils,Percentage FSM,Street,Locality,Town,Postcode,Ofsted Rating,LSOA (code),FSM
91,100163,Greenwich,Mulgrave Primary School,Community school,Local authority maintained schools,3,11,397,24.9,Rectory Place,Woolwich,London,SE18 5DL,Good,E01001705,93
104,100177,Greenwich,St Thomas A Becket Roman Catholic Primary School,Voluntary aided school,Local authority maintained schools,3,11,429,29.2,Mottisfont Road,Abbey Wood,London,SE2 9LY,Good,E01001579,113
109,100182,Greenwich,Eltham Hill School,Community school,Local authority maintained schools,11,19,1227,20.8,Eltham Hill,NaN,London,SE9 5EE,Good,E01001655,203
117,100202,Greenwich,Colfe's School,Other independent school,Independent schools,3,19,1280,0.0,Horn Park Lane,Lee,London,SE12 8AW,NaN,E01001659,0
161,100279,Hackney,Stoke Newington School and Sixth Form,Community school,Local authority maintained schools,11,19,1708,32.0,Clissold Road,NaN,London,N16 9EX,Good,E01001737,405


In [27]:
#Cardinality = Main Table:DfE table (Many:One), duplicates in Table 1, no duplicates in Table 2 - Keep all data in the main, only match data in right
Att_DfE = Att_v1.merge(DfE_matched_df, how="left", on="URN", indicator=True)

#Check New Merged shape with 15/16 selected columns added from the DfE Index
print(Att_DfE.shape)

#Alternative joining code
#df1 = pd.DataFrame(Att_v1)
#df2 = pd.DataFrame(DfE_matched_df)
#Att_DfE = df1.merge(df2, how='left', on="URN", indicator=True)
#Att_DfE

#Count how many rows matched vs did not match
uniqueurn_match_counts = Att_DfE["_merge"].value_counts()

#Calculate match percentage
uniqueurnmatch_rate = (uniqueurn_match_counts.get("both", 0) / len(Att_DfE)) * 100

print("Unique URN Match counts:")
print(uniqueurn_match_counts)
print(f"\nMatch rate: {uniqueurnmatch_rate:.2f}% of school records found in the DfE Index")

Att_DfE.head()

(11425, 60)
Unique URN Match counts:
_merge
both          6624
left_only     4801
right_only       0
Name: count, dtype: int64

Match rate: 57.98% of school records found in the DfE Index


,Record_ID,Event,STEAM Season,Partner,Event Sub-Type,Year,Year-Quarter-Month,Type,Quantity,URN_1,Region,LSOA (%),State-funded (%),FSM (%),Age Today,Stream,Age on Day of,P/G Attend Uni,P/G Work in STEAM,Pronouns,Ethnic Background,Physical / Virtual,UK-Based,Ethnic Background (Grouped),Total Students Engaged,How Did You Hear,Where Did You Hear,QQ,Non-UK based country,Record_ID (Onb),URN_2,URN_3,URN_4,Onboarded Date/Time,First Time Attendee? (Y/N),Record_ID (Unique),Sign-up Date/Time,STEAM Tool,Record_ID (Fb),# Fb,Fb sub date,Fb event date,Date/Time (1st Sign-up),URN,Local Authority Name,Establishment Name,Type of Establishment,Establishment Type Group,Statutory_Low_Age,Statutory_High_Age,No. of Pupils,Percentage FSM,Street,Locality,Town,Postcode,Ofsted Rating,LSOA (code),FSM,_merge
0,recUrZnORsIVbVFwr,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,Region TBC,LSOA TBC,1.0,100%,17.0,Academy,16.0,No,No,She/ Her,Black or Black British - African,I will be joining in-person,Yes,Black or Black British,NaN,Through School / College / Uni,NaN,NaN,NaN,recvL2l853VHDlFzP,NaN,NaN,<NA>,11/7/2025 9:35am,EXISTING/Repeat Attendee,recMWiC7pw1ZO0hfe,10/7/2025 4:26pm,NaN,recwapMx1jjXPUZlp,1,19/7/2025,19/7/2025,10/7/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,left_only
1,recqe2XUQ0eSk0CTf,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,London,4,1.0,100%,17.0,Academy,17.0,No,Yes,She/ Her,Mixed - White and Black African,I will be joining in-person,Yes,Mixed,NaN,Stemette Newsletter,NaN,NaN,NaN,rechqToGPUTdlChai,NaN,NaN,<NA>,12/5/2025 2:37pm,NEW/First-time Attendee,recrPMUI4ZCTOkgop,30/4/2025 9:01pm,NaN,NaN,0,NaN,NaN,30/4/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,left_only
2,recUdaEr0T20VEL5N,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,Region TBC,LSOA TBC,1.0,100%,19.0,Academy,19.0,Yes,Yes,She/ Her,Other,I will be joining in-person,Yes,Other,NaN,"Different Stemettes Event/Programme,Other",Broadening Horizons,NaN,NaN,recNh4wM6zqJ4SNDT,NaN,NaN,<NA>,18/6/2025 10:39am,EXISTING/Repeat Attendee,rec8fz0wx3s9gxbuH,18/6/2025 9:29am,NaN,NaN,0,NaN,NaN,26/6/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,left_only
3,recap8t8pYlhK4O7D,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,Region TBC,LSOA TBC,1.0,NaN,17.0,Academy,17.0,Yes,Yes,She/ Her,Black or Black British - Caribbean,I will be joining in-person,Yes,Black or Black British,NaN,Through School / College / Uni,NaN,NaN,NaN,recAcWLRHTgZeNxcs,NaN,NaN,<NA>,10/7/2025 9:55pm,NEW/First-time Attendee,recsamxoNTI98650L,9/7/2025 12:08pm,NaN,recw20mMvMLX0hJp7,1,19/7/2025,19/7/2025,9/7/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,left_only
4,recnGwj1cxBF7ihZy,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,Region TBC,LSOA TBC,NaN,NaN,16.0,Academy,16.0,Yes,Yes,She/ Her,Asian or Asian British - Other,I will be joining in-person,Yes,Asian or Asian British,NaN,LinkedIn,NaN,NaN,NaN,recPoBoCFPhuWXBo4,NaN,NaN,<NA>,2/6/2025 11:49am,NEW/First-time Attendee,recbcAvydYk5DrsEm,1/6/2025 1:39pm,NaN,rec09fGZ9A55CgoJ7,1,19/7/2025,19/7/2025,1/6/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,left_only


In [28]:
# (2.10) Make a copy before making any modifications to the concatenated and merged data frame
Att_new2 = Att_DfE.copy()

#Convert joined list to dataframe
Att_v2 = pd.DataFrame(Att_new2)
Att_v2.head()

#Check New Merged shape with 15/16 selected columns added from the DfE Index
print(Att_v2.shape)

#Generate Summary Stats for Numerical Data
##Descriptive Stats for different time-periods, streams, regions, types... maybe boss- repeat this process for sign-ups
Att_v2.describe()

(11425, 60)


,Year,Quantity,State-funded (%),Age Today,Age on Day of,Total Students Engaged,# Fb,Statutory_Low_Age,Statutory_High_Age,No. of Pupils,Percentage FSM,FSM
count,11425.000000,11425.0,9957.000000,1237.000000,2432.000000,3006.000000,11425.000000,6484.0,6484.0,6484.0,6484.000000,6484.0
mean,2023.595536,127.137418,0.904991,15.413096,13.112664,18.536593,0.073961,8.175663,15.462523,800.192474,23.325200,152.896977
std,0.605036,198.288198,0.293241,5.092979,3.841423,16.801112,0.330322,3.757063,3.178185,497.437269,15.747513,132.055285
min,2023.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,7.0,0.0,0.000000,0.0
25%,2023.000000,10.0,1.000000,12.000000,11.000000,10.000000,0.000000,4.0,11.0,393.0,10.000000,33.0
50%,2024.000000,10.0,1.000000,16.000000,12.000000,10.000000,0.000000,11.0,16.0,704.0,24.500000,105.0
75%,2024.000000,200.0,1.000000,18.000000,15.000000,28.000000,0.000000,11.0,18.0,1163.0,33.100000,277.0
max,2025.000000,650.0,1.000000,63.000000,63.000000,90.000000,5.000000,16.0,25.0,2990.0,77.600000,660.0


In [29]:
#Check how many values in the Record_ID column are Unique (no duplicates)
print("Total Att Records:", Att_v2['Record_ID'].nunique()) #Print no. of unique entries

#Check how many values in the Record_ID column are NOT Unique (contain duplicates)
print("Duplicate Total Att Records:", Att_v2['Record_ID'].value_counts().max())

#Check how many values in the Record_ID (Unique) column are Unique (no duplicates)
print("Unique Att Records:", Att_v2['Record_ID (Unique)'].nunique()) #Print no. of unique entries

#Check how many values in the Record_ID (Unique) column are NOT Unique (contain duplicates)
print("Duplicate Unique Att Records:", Att_v2['Record_ID (Unique)'].value_counts().max())

Total Att Records: 11425
Duplicate Total Att Records: 1
Unique Att Records: 929
Duplicate Unique Att Records: 14


## 🔍 3.0 Understanding the Contacts Data

This table contains thousands of records, each representing an individual young person (contact). Each record captures how a person moves through the engagement funnel (sign-up → eligibility/invite → onboarding → attendance), plus demographics, attribution, and communication preferences. In total there are 61 variables that let you link people to programmes and events, measure participation over time, and understand who you’re reaching and how.

At a glance, the fields cover:
- Identifiers & linkage: names, email, Record_ID (Contacts), Record_ID (Sign-ups), cross-table join keys (e.g., Contacts × Sign-ups, Programmes × Att), and dedupe helpers like Unique Name / Unique Event.
- Engagement funnel & counts: signed up / eligible & invited / onboarded / attended flags, # Sign-ups, # Onb, # Att, First Time Attendee?, Attended 1< ?, badges/stamps, points, and # FB (feedbacks).
- Programme & timing context: Year/Month/Quarter (YY, QQ, “MM# MMM”), start dates, first sign-up created time, SSphere sign-up timestamp, and links back to programmes/events and streams.
- Demographics & eligibility: Age Today, Date_of_Birth, UK Region, Ethnicity / Ethnic_Background, and FSM_Eligibility.
- Attribution (how they heard): free-text “Where/How did you hear…”, plus channel flags (Social Media, Girlguiding, Schools, Stemettes, Parent/Friend, Other).
- Comms & consent: Email Type, Opt-in to receive emails (Y/N), and community flags (Sign up for SSoc?, In SSoc/SFJB/SFYB?).

Together, these variables let you analyse conversion at each funnel stage, retention (first-time vs repeat), audience segments (region, age, FSM/ethnicity), channel performance (where contacts came from), and timing patterns across months and quarters—while keeping records properly linked to programmes and events.

In [30]:
# (3.1) Import Exported Unique Participation Data
t7= pd.read_csv('Data/Unique Pp.csv')

#Print raw shape of the imported table
print("Raw Shape:", t7.shape)

Raw Shape: (5821, 61)


In [31]:
# (3.2) Get a numbered index list of all unique values in UniqPp 'field_name' column to decide on columns of interest for later slicing/ grouping
for i, col in enumerate(t7.columns):
    print(f"{i}: {col} ({t7[col].dtype})")

0: Name (object)
1: FULL Name (object)
2: Record_ID (Contacts) (object)
3: Record_ID (Sign-ups) (object)
4: Contacts x Sign-ups (object)
5: (1) Signed up for [ALL Events] (object)
6: # Sign-ups (int64)
7: (2) Eligible & Invited to Attend [Events] (object)
8: (3) Onboarded [ALL} [Events] (object)
9: # Onb (int64)
10: (4) Onboarded & Attended [Events] (object)
11: 1st Eligible/Attended Event ? (object)
12: Attended (Y/N) (from Onboarded) (from Eligibility x Contacts) (from Contacts x Sign-ups) (object)
13: # Att (int64)
14: Attended 1< ? (int64)
15: First Time Attendee? (Y/N) (object)
16: Badges/ Stamps (object)
17: Points (object)
18: # FB (int64)
19: Contacts x Programmes (float64)
20: Programmes x Att (object)
21: Year (from Programmes) (object)
22: Month No. (from Programmes) (object)
23: Eligibility (float64)
24: Preferences (float64)
25: trigger (object)
26: Gateway Event (object)
27: Start Date (from Programmes) (object)
28: Where did you hear about this event? (object)
29: Age To

In [32]:
# (3.3a) Define the t7 column rename mapping using indices to make grouping easier later
t7_column_mapping = {
    t7.columns[2]: "Record_ID (Unique)",
    t7.columns[6]: "# Sign-ups",
    t7.columns[9]: "# Onb",
    t7.columns[12]: "Att (Y/N)",
    t7.columns[13]: "# Att",
    t7.columns[14]: "Att 1+",
    t7.columns[15]: "First Time Attendee? (Y/N)",
    t7.columns[17]: "Points",
    t7.columns[18]: "# Feeback",
    t7.columns[28]: "Where_heard_about",
    t7.columns[29]: "Age Today",
    t7.columns[45]: "SSphere Sign-up Date/Time",
    t7.columns[53]: "Email Opt-in (Y/N) ",
    t7.columns[58]: "In SSoc?",
    t7.columns[59]: "In SFJB?",
    t7.columns[60]: "In SFYB?",
}

# (3.3b) Apply the Main Table renaming to all four attendance tables
t7 = t7.rename(columns=t7_column_mapping)

# (3.3c) Get the list of new t7 column names (values from your mapping)
renamed_t7_columns = list(t7_column_mapping.values())

print("Renaming:", t7_column_mapping)

Renaming: {'Record_ID (Contacts)': 'Record_ID (Unique)', '# Sign-ups': '# Sign-ups', '# Onb': '# Onb', 'Attended (Y/N) (from Onboarded) (from Eligibility x Contacts) (from Contacts x Sign-ups)': 'Att (Y/N)', '# Att': '# Att', 'Attended 1< ?': 'Att 1+', 'First Time Attendee? (Y/N)': 'First Time Attendee? (Y/N)', 'Points': 'Points', '# FB': '# Feeback', 'Where did you hear about this event?': 'Where_heard_about', 'Age Today': 'Age Today', 'SSphere Sign-up Date/Time': 'SSphere Sign-up Date/Time', 'Opt-in to receive emails (Y/N)': 'Email Opt-in (Y/N) ', '*In SSoc?': 'In SSoc?', '*In SFJB?': 'In SFJB?', '*In SFYB?': 'In SFYB?'}


In [33]:
# (3.4) Build a DataFrame showing t7 old name vs new name
t7_rename_comparison = pd.DataFrame({
    "T7 Original": list(t7_column_mapping.keys()),
    "T7 Renamed": list(t7_column_mapping.values())
})

#Show the table directly
t7_rename_comparison

from IPython.display import display, HTML

#Nicely formatted HTML version
display(HTML(f"""
    <div style="flex: 1;">
        <h3>Unique Participation Data (T7) Column Mapping</h3>
        {t7_rename_comparison.to_html(index=False)}
    </div>
"""))

T7 Original,T7 Renamed
Record_ID (Contacts),Record_ID (Unique)
# Sign-ups,# Sign-ups
# Onb,# Onb
Attended (Y/N) (from Onboarded) (from Eligibility x Contacts) (from Contacts x Sign-ups),Att (Y/N)
# Att,# Att
Attended 1< ?,Att 1+
First Time Attendee? (Y/N),First Time Attendee? (Y/N)
Points,Points
# FB,# Feeback
Where did you hear about this event?,Where_heard_about


In [34]:
# (3.5a) Use only the renamed columns from each dataframe
UniqPp = t7[renamed_t7_columns]

# (3.5b) Concatenate the Renamed DfE Tables into 1 - #No need to concat
#UniqPp = pd.concat([t7_renamed], axis=0)

# (3.5c) #Validate transformation process- Check the before/after shape of the concatenated Dataframe - #No need to validate
#print("Rows before:", len(t7_renamed), "| Rows after:", len(UniqPp))
print(UniqPp.shape)

#Check that all the values in the Record_ID column are Unique (no duplicates) - #No need to check for duplicates
#print(UniqPp['Record_ID'].value_counts().max())

#Check that all the values in the Record_ID (Unique) column are not Unique (contain duplicates) - #No need to check for duplicate values
#print(UniqPp['Record_ID (Unique)'].value_counts().max())

#Remove Index and use Record_ID as Primary field
#UniqPp = UniqPp.set_index('Record_ID')

#Optional Preview of the first 3 rows only
UniqPp.head(3)

#--

## (3.5b) Concatenate the 4x Renamed Attendance Tables into 1
#UniqPp = pd.concat([Att, t7_renamed], axis=0)

## (3.5c) #Validate transformation process- Check the before/after shape of the concatenated Dataframe
#print("Rows before:", len(Att), len(t7_renamed), "| Rows after:", len(Att))
#print(Att.shape)

##Check that all the values in the Record_ID column are Unique (no duplicates)
#print(Att['Record_ID'].value_counts().max())

##Check that all the values in the Record_ID (Unique) column are not Unique (contain duplicates)
#print(Att['Record_ID (Unique)'].value_counts().max())

##Remove Index and use Record_ID as Primary field
##Att = Att.set_index('Record_ID')

##Optional Preview of the first 3 rows only
#Att.head(3)

##Convert expanded list to dataframe????????
##Att = pd.DataFrame(Att_exp)

(5821, 16)


,Record_ID (Unique),# Sign-ups,# Onb,Att (Y/N),# Att,Att 1+,First Time Attendee? (Y/N),Points,# Feeback,Where_heard_about,Age Today,SSphere Sign-up Date/Time,Email Opt-in (Y/N),In SSoc?,In SFJB?,In SFYB?
0,recRHLZAKYvDJbMNC,1,1,1 checked out of 1,1,1,NEW/First-time Attendee,NaN,2,"Dream, Build, Play with Trolls @ Lenovo",10.0,15/1/2024,0 checked out of 1,NaN,NaN,NaN
1,rec9zLYUYmpMNXJRZ,1,0,NaN,0,0,GHOST/Not Yet Att,NaN,0,NaN,NaN,17/12/2023,0 checked out of 1,NaN,NaN,NaN
2,rec1XFfOIji3SyoIm,1,1,1 checked out of 1,1,1,NEW/First-time Attendee,NaN,1,"Dream, Build, Play with Trolls @ Lenovo",10.0,15/1/2024,0 checked out of 1,NaN,NaN,NaN


In [35]:
# (3.6) Make a copy of the newly concatenated df for easy referencing to the original (WITHOUT ANY columns that should remain anonymous i.e. AFTER we dropped the Person column)
#UniqPp_anon = UniqPp.drop(columns=["_merge"])
UniqPp_anon = UniqPp.copy() #Use if I have no columns to drop

#Check the new shape of the concatenated and anonymised Dataframe
print(UniqPp_anon.shape)

#Preview the concatenated and anonymous dataframe
UniqPp_anon.head(3)

(5821, 16)


,Record_ID (Unique),# Sign-ups,# Onb,Att (Y/N),# Att,Att 1+,First Time Attendee? (Y/N),Points,# Feeback,Where_heard_about,Age Today,SSphere Sign-up Date/Time,Email Opt-in (Y/N),In SSoc?,In SFJB?,In SFYB?
0,recRHLZAKYvDJbMNC,1,1,1 checked out of 1,1,1,NEW/First-time Attendee,NaN,2,"Dream, Build, Play with Trolls @ Lenovo",10.0,15/1/2024,0 checked out of 1,NaN,NaN,NaN
1,rec9zLYUYmpMNXJRZ,1,0,NaN,0,0,GHOST/Not Yet Att,NaN,0,NaN,NaN,17/12/2023,0 checked out of 1,NaN,NaN,NaN
2,rec1XFfOIji3SyoIm,1,1,1 checked out of 1,1,1,NEW/First-time Attendee,NaN,1,"Dream, Build, Play with Trolls @ Lenovo",10.0,15/1/2024,0 checked out of 1,NaN,NaN,NaN


In [36]:
# (3.7) Apply DataType Transformations to Anonymous Dataset
#Change Age Today to Integer- this will be 'key' (matching column) for merge
UniqPp_anon.loc[:, 'Age Today'] = pd.to_numeric(UniqPp_anon['Age Today'], errors='coerce').astype('Int64')

#Convert date fields to ISO 8:
UniqPp_anon['SSphere Sign-up Date/Time'] = pd.to_datetime(UniqPp_anon['SSphere Sign-up Date/Time'], errors='coerce', dayfirst=True)

#Make a copy after applying transformations to concatenated dataset, before creating any slices of the original data
UniqPp_tf = UniqPp_anon.copy()

#Check the new shape of the transformed Dataframe
print(UniqPp_tf.shape)

#Preview the result
UniqPp_tf.head(3)

(5821, 16)


,Record_ID (Unique),# Sign-ups,# Onb,Att (Y/N),# Att,Att 1+,First Time Attendee? (Y/N),Points,# Feeback,Where_heard_about,Age Today,SSphere Sign-up Date/Time,Email Opt-in (Y/N),In SSoc?,In SFJB?,In SFYB?
0,recRHLZAKYvDJbMNC,1,1,1 checked out of 1,1,1,NEW/First-time Attendee,NaN,2,"Dream, Build, Play with Trolls @ Lenovo",10,2024-01-15,0 checked out of 1,NaN,NaN,NaN
1,rec9zLYUYmpMNXJRZ,1,0,NaN,0,0,GHOST/Not Yet Att,NaN,0,NaN,<NA>,2023-12-17,0 checked out of 1,NaN,NaN,NaN
2,rec1XFfOIji3SyoIm,1,1,1 checked out of 1,1,1,NEW/First-time Attendee,NaN,1,"Dream, Build, Play with Trolls @ Lenovo",10,2024-01-15,0 checked out of 1,NaN,NaN,NaN


In [37]:
# (3.8) Add any new columns for calculations
#Make a copy before adding a new column to the transformed dataframe
UniqPp_new = UniqPp_tf.copy()

#Create a new column called x that does y for each row
#UniqPp_new['x'] = UniqPp_new[['y']].bfill(axis=1).iloc[:, 0]

#Check the new shape of the Dataframe
print(UniqPp_new.shape)

#Preview the result
UniqPp_new.head()

(5821, 16)


,Record_ID (Unique),# Sign-ups,# Onb,Att (Y/N),# Att,Att 1+,First Time Attendee? (Y/N),Points,# Feeback,Where_heard_about,Age Today,SSphere Sign-up Date/Time,Email Opt-in (Y/N),In SSoc?,In SFJB?,In SFYB?
0,recRHLZAKYvDJbMNC,1,1,1 checked out of 1,1,1,NEW/First-time Attendee,NaN,2,"Dream, Build, Play with Trolls @ Lenovo",10,2024-01-15,0 checked out of 1,NaN,NaN,NaN
1,rec9zLYUYmpMNXJRZ,1,0,NaN,0,0,GHOST/Not Yet Att,NaN,0,NaN,<NA>,2023-12-17,0 checked out of 1,NaN,NaN,NaN
2,rec1XFfOIji3SyoIm,1,1,1 checked out of 1,1,1,NEW/First-time Attendee,NaN,1,"Dream, Build, Play with Trolls @ Lenovo",10,2024-01-15,0 checked out of 1,NaN,NaN,NaN
3,recDs8aaoPSt0PqzD,1,1,0 checked out of 1,0,0,GHOST/Not Yet Att,NaN,0,NaN,15,2025-01-21,1 checked out of 1,NaN,NaN,NaN
4,recST5EcqpzRqtvB2,1,0,NaN,0,0,GHOST/Not Yet Att,NaN,0,NaN,14,2025-07-29,1 checked out of 1,NaN,NaN,NaN


In [38]:
# (3.9) Misc
UniqPp_matched = UniqPp_new[UniqPp_new['Record_ID (Unique)'].notna() & UniqPp_new['Record_ID (Unique)'].isin(Att_v2['Record_ID (Unique)'].dropna())]

#Convert t7_matched list to dataframe
UniqPp_matched_df = pd.DataFrame(UniqPp_matched)

#Check number of schools from Unique Participation that appear in the consolidated attendance table
print("Unique Attendance Shape (pre-concat):", UniqPp_matched_df.shape)
UniqPp_matched_df.head()

#Drop previous "_merge" column
for df in (Att_v2, UniqPp_matched_df):
    df.drop(columns=["_merge"], errors="ignore", inplace=True)

#Cardinality = Main Table:DfE table (Many:One), duplicates in Table 1, no duplicates in Table 2 - Keep all data in the main, only match data in right
Att_UniqPp = Att_v2.merge(UniqPp_matched_df, how="left", on="Record_ID (Unique)", indicator=True)

#Check New Merged shape with 15/16 selected columns added from the DfE Index
print("Unique Attendance Shape (post-concat):", Att_UniqPp.shape)

#Alternative joining code
#df1 = pd.DataFrame(Att_v2)
#df2 = pd.DataFrame(UniqPp_matched_df)
#Att_UniqPp = df1.merge(df2, how='left', on="Record_ID (Unique)", indicator=True)
#Att_UniqPp

#Count how many rows matched vs did not match
uniqueatt_match_counts = Att_UniqPp["_merge"].value_counts()

#Calculate match percentage
uniqueatt_match_rate = (uniqueatt_match_counts.get("both", 0) / len(Att_UniqPp)) * 100

print("Unique Attendance Match counts:")
print(uniqueatt_match_counts)
print(f"\nMatch rate: {uniqueatt_match_rate:.2f}% of unique participant records found in the Attendance Table")

Att_UniqPp.head()

Unique Attendance Shape (pre-concat): (928, 16)
Unique Attendance Shape (post-concat): (11425, 75)
Unique Attendance Match counts:
_merge
left_only     10189
both           1236
right_only        0
Name: count, dtype: int64

Match rate: 10.82% of unique participant records found in the Attendance Table


,Record_ID,Event,STEAM Season,Partner,Event Sub-Type,Year,Year-Quarter-Month,Type,Quantity,URN_1,Region,LSOA (%),State-funded (%),FSM (%),Age Today_x,Stream,Age on Day of,P/G Attend Uni,P/G Work in STEAM,Pronouns,Ethnic Background,Physical / Virtual,UK-Based,Ethnic Background (Grouped),Total Students Engaged,How Did You Hear,Where Did You Hear,QQ,Non-UK based country,Record_ID (Onb),URN_2,URN_3,URN_4,Onboarded Date/Time,First Time Attendee? (Y/N)_x,Record_ID (Unique),Sign-up Date/Time,STEAM Tool,Record_ID (Fb),# Fb,Fb sub date,Fb event date,Date/Time (1st Sign-up),URN,Local Authority Name,Establishment Name,Type of Establishment,Establishment Type Group,Statutory_Low_Age,Statutory_High_Age,No. of Pupils,Percentage FSM,Street,Locality,Town,Postcode,Ofsted Rating,LSOA (code),FSM,# Sign-ups,# Onb,Att (Y/N),# Att,Att 1+,First Time Attendee? (Y/N)_y,Points,# Feeback,Where_heard_about,Age Today_y,SSphere Sign-up Date/Time,Email Opt-in (Y/N),In SSoc?,In SFJB?,In SFYB?,_merge
0,recUrZnORsIVbVFwr,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,Region TBC,LSOA TBC,1.0,100%,17.0,Academy,16.0,No,No,She/ Her,Black or Black British - African,I will be joining in-person,Yes,Black or Black British,NaN,Through School / College / Uni,NaN,NaN,NaN,recvL2l853VHDlFzP,NaN,NaN,<NA>,11/7/2025 9:35am,EXISTING/Repeat Attendee,recMWiC7pw1ZO0hfe,10/7/2025 4:26pm,NaN,recwapMx1jjXPUZlp,1,19/7/2025,19/7/2025,10/7/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,2.0,2.0,2 checked out of 2,2.0,1.0,EXISTING/Repeat Attendee,"10, 40",2.0,NaN,17,2025-07-10,0 checked out of 1,NaN,NaN,NaN,both
1,recqe2XUQ0eSk0CTf,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,London,4,1.0,100%,17.0,Academy,17.0,No,Yes,She/ Her,Mixed - White and Black African,I will be joining in-person,Yes,Mixed,NaN,Stemette Newsletter,NaN,NaN,NaN,rechqToGPUTdlChai,NaN,NaN,<NA>,12/5/2025 2:37pm,NEW/First-time Attendee,recrPMUI4ZCTOkgop,30/4/2025 9:01pm,NaN,NaN,0,NaN,NaN,30/4/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,1.0,2.0,1 checked out of 2,1.0,1.0,NEW/First-time Attendee,10,1.0,NaN,17,2025-04-30,0 checked out of 1,NaN,NaN,NaN,both
2,recUdaEr0T20VEL5N,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,Region TBC,LSOA TBC,1.0,100%,19.0,Academy,19.0,Yes,Yes,She/ Her,Other,I will be joining in-person,Yes,Other,NaN,"Different Stemettes Event/Programme,Other",Broadening Horizons,NaN,NaN,recNh4wM6zqJ4SNDT,NaN,NaN,<NA>,18/6/2025 10:39am,EXISTING/Repeat Attendee,rec8fz0wx3s9gxbuH,18/6/2025 9:29am,NaN,NaN,0,NaN,NaN,26/6/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,4.0,4.0,1 checked out of 4,1.0,1.0,NEW/First-time Attendee,10,0.0,NaN,19,2025-06-26,0 checked out of 1,NaN,NaN,NaN,both
3,recap8t8pYlhK4O7D,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,Region TBC,LSOA TBC,1.0,NaN,17.0,Academy,17.0,Yes,Yes,She/ Her,Black or Black British - Caribbean,I will be joining in-person,Yes,Black or Black British,NaN,Through School / College / Uni,NaN,NaN,NaN,recAcWLRHTgZeNxcs,NaN,NaN,<NA>,10/7/2025 9:55pm,NEW/First-time Attendee,recsamxoNTI98650L,9/7/2025 12:08pm,NaN,recw20mMvMLX0hJp7,1,19/7/2025,19/7/2025,9/7/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,1.0,1.0,1 checked out of 1,1.0,1.0,NEW/First-time Attendee,10,1.0,NaN,17,2025-07-09,0 checked out of 1,NaN,NaN,NaN,both
4,recnGwj1cxBF7ihZy,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,Region TBC,LSOA TBC,NaN,NaN,16.0,Aca

In [39]:
# (3.10) Make a copy before making any modifications to the concatenated and merged data frame
Att_new3 = Att_UniqPp.copy()

#Convert joined list to dataframe
Att_v3 = pd.DataFrame(Att_new3)
Att_v3.head()

#Check New Merged shape with 15/16 selected columns added from the DfE Index
print(Att_v3.shape)

# Generate Summary Stats for Numerical Data
##Descriptive Stats for different time-periods, streams, regions, types... maybe boss- repeat this process for sign-ups
Att_v3.describe()

(11425, 75)


,Year,Quantity,State-funded (%),Age Today_x,Age on Day of,Total Students Engaged,# Fb,Statutory_Low_Age,Statutory_High_Age,No. of Pupils,Percentage FSM,FSM,# Sign-ups,# Onb,# Att,Att 1+,# Feeback,Age Today_y,SSphere Sign-up Date/Time
count,11425.000000,11425.0,9957.000000,1237.000000,2432.000000,3006.000000,11425.000000,6484.0,6484.0,6484.0,6484.000000,6484.0,1236.000000,1236.000000,1236.000000,1236.000000,1236.000000,1233.0,1236
mean,2023.595536,127.137418,0.904991,15.413096,13.112664,18.536593,0.073961,8.175663,15.462523,800.192474,23.325200,152.896977,4.127832,3.652104,2.766990,0.991909,2.322006,15.30738,2024-10-28 03:51:50.679611648
min,2023.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,7.0,0.0,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,2023-03-24 00:00:00
25%,2023.000000,10.0,1.000000,12.000000,11.000000,10.000000,0.000000,4.0,11.0,393.0,10.000000,33.0,1.000000,1.000000,1.000000,1.000000,0.000000,12.0,2024-05-16 00:00:00
50%,2024.000000,10.0,1.000000,16.000000,12.000000,10.000000,0.000000,11.0,16.0,704.0,24.500000,105.0,2.000000,2.000000,1.000000,1.000000,1.000000,15.0,2024-10-30 12:00:00
75%,2024.000000,200.0,1.000000,18.000000,15.000000,28.000000,0.000000,11.0,18.0,1163.0,33.100000,277.0,5.000000,4.000000,3.000000,1.000000,3.000000,18.0,2025-04-20 06:00:00
max,2025.000000,650.0,1.000000,63.000000,63.000000,90.000000,5.000000,16.0,25.0,2990.0,77.600000,660.0,28.000000,26.000000,20.000000,1.000000,25.000000,63.0,2025-08-28 00:00:00
std,0.605036,198.288198,0.293241,5.092979,3.841423,16.801112,0.330322,3.757063,3.178185,497.437269,15.747513,132.055285,5.311038,4.637674,3.363882,0.089620,3.840530,5.135541,NaN


In [40]:
#Check how many values in the Record_ID column are Unique (no duplicates)
print("Total Att Records:", Att_v3['Record_ID'].nunique()) #Print no. of unique entries

#Check how many values in the Record_ID column are NOT Unique (contain duplicates)
print("Duplicate Total Att Records:", Att_v3['Record_ID'].value_counts().max())

#Check how many values in the Record_ID (Unique) column are Unique (no duplicates)
print("Unique Att Records:", Att_v3['Record_ID (Unique)'].nunique()) #Print no. of unique entries

#Check how many values in the Record_ID (Unique) column are NOT Unique (contain duplicates)
print("Duplicate Unique Att Records:", Att_v3['Record_ID (Unique)'].value_counts().max())

Total Att Records: 11425
Duplicate Total Att Records: 1
Unique Att Records: 929
Duplicate Unique Att Records: 14


## 🔍 4.0 Understanding the Feedback Data

This table contains hundreds of records, each representing feedback from a young person/student about a specific Stemettes event/programme. The 250 variables associated with each record provide rich metadata linking the response to programmes and people (e.g., Record IDs, dates, streams, seasons), as well as a wide mix of quantitative ratings (often 0–10 scales), qualitative open-text answers, engagement and intention indicators, and baseline vs. post-event uplift measures.

At a high level, the variables cover:
- Identifiers & linkage: Feedback/Eligibility/Onboarding/Preferences Record IDs, timestamps, programme/event references, contact fields, and de-duplication helpers.
- Programme context: Stream, STEAM Season, Year/Quarter/Month, Event Type/Sub-Type, Partner, Target Demographic, Eligible Age Group, modality (physical/virtual).
- Participant profile: Age on day/today, age group/stream by age, UK region, contact type, (where present) school name and counts tied to initiatives (e.g., Stemillions).
- Experience & content ratings: Multiple /10 scores (experience, enjoyment, content), plus free-text on most/least favourite parts and challenges.
- STEAM outcomes (pre/post): Perception, awareness, network, confidence, knowledge—primary flags, improvement indicators, and weighted uplift measures.
- Wellbeing & skills: Mental wellbeing, self-confidence, creativity importance/contributors, skills improved, MS Teams difficulty, usefulness of guides.
- Intentions & pathways: Pre/post intentions (STEAM/non-STEAM/uncertain), likelihood to consider STEAM, future participation, other opportunities in/outside Stemettes, mentorship follow-ups.
- Operational details: Attendance flags (preferences vs. feedback), counts, QR/link helpers, created-by fields, exclusions, resource design/engagement markers.

Together, these fields let you analyse participant experience and learning, uplift from baseline, who you’re reaching and how, and which formats/streams/partners drive the biggest impact.

In [41]:
# (4.1) Import Exported Feedback Data
t8= pd.read_csv('Data/Table 4-2025fb.csv') 
t8.head(1)

#Print raw shape of the imported table
print("Raw Shape:", t8.shape)

Raw Shape: (714, 250)


In [42]:
# (4.2) Get a numbered index list of all unique values in main table > (t8) 'field_name' column to decide on columns of interest for later slicing/ grouping
for i, col in enumerate(t8.columns):
    print(f"{i}: {col} ({t8[col].dtype})")

0: Full Name (object)
1: Record_ID (Feedback) (object)
2: Programmes x FB (object)
3: STEAM Seasons (from Programmes x FB) 2 (object)
4: Year-Quarter-Month (from Programmes x FB) (object)
5: Start Date (object)
6: Date FB Submitted (object)
7: Record_ID (Eligibility) (object)
8: Record_ID (Onboarding) (object)
9: Preferences x FB (object)
10: FB x Stage (object)
11: Eligibility x FB (object)
12: *Contact*_FirstName (from *Contacts x Eligibility) (from Eligibility x FB) (object)
13: FIRST name (manual) (object)
14: LAST name (manual) (object)
15: FULL name (manual) (object)
16: Email (manual) (object)
17: Age on day of event (from *Contacts x Eligibility) (from Eligibility x FB) (float64)
18: What are your preferences for attending? (from Contacts x Eligibility) (from Eligibility x FB) (object)
19: Stream (object)
20: Age Group (object)
21: Stream (Based on Age Today) (object)
22: UK Region (from *Contacts x Eligibility) (from Eligibility x FB) (object)
23: Out of 10,  how would you rat

In [43]:
# (4.3a) Define the column rename mapping using indices to make grouping easier later
t8_column_mapping = {
    t8.columns[1]: "Record_ID (Feedback)",
    t8.columns[2]: "Event",
    t8.columns[5]: "Start Date", #Datetime64
    t8.columns[8]: "Record_ID (Onb)", #for matching fb to onb&att record
    t8.columns[10]: "t8 Stage",
    t8.columns[19]: "Stream_1", #Based on Age on Day of Event?
    t8.columns[11]: "Age Group", #Based on Age Today?
    t8.columns[21]: "Stream_2", #Based on Age Today
    t8.columns[22]: "Region",
    t8.columns[25]: "Experience /10",
    t8.columns[34]: "Content /10",
    t8.columns[39]: "Inn & PS Imp",
    t8.columns[40]: "Pre-event STEM Vs. STEAM Conn",
    t8.columns[41]: "STEAM Separate?",
    t8.columns[42]: "Post-event STEM Vs. STEAM CONNECTION",
    t8.columns[47]: "Conn Imp",
    t8.columns[50]: "Perception Imp", #"P" in "PANCK"
    t8.columns[48]: "STEAM Perception_After", #"P" in "PANCK"
    t8.columns[51]: "STEAM Awareness_After", #"A" in "PANCK"
    t8.columns[54]: "STEAM Network_After", #"N" in "PANCK"
    t8.columns[57]: "STEAM Confidence_After", #"C" in "PANCK"
    t8.columns[60]: "STEAM Knowledge_After", #"K" in "PANCK"
    t8.columns[69]: "STEAM Well-being_After", #"Wb"
    t8.columns[53]: "Awareness Imp", #"A" in "PANCK"
    t8.columns[56]: "Network Imp", #"N" in "PANCK"
    t8.columns[59]: "Confidence Imp", #"C" in "PANCK"
    t8.columns[62]: "Knowledge Imp", #"K" in "PANCK"
    t8.columns[71]: "Well-being Imp", #Well-being/ Better mood
    t8.columns[73]: "Self-Confidence Imp", #Confidence in self not abilites
    t8.columns[75]: "Creativity Imp",
    t8.columns[80]: "Pre-event Intentions", #Combined Primary & Secondary (both)
    t8.columns[81]: "Pre STEAM",
    t8.columns[82]: "Pre non-STEAM",
    t8.columns[83]: "Pre Uncertain",
    t8.columns[96]: "Post-event Intentions", #Combined Primary & Secondary (both)
    t8.columns[97]: "STEAM (All)", #Combined Primary & Secondary (both)
    t8.columns[100]: "Likelihood (Both)",
    t8.columns[108]: "Team Name", #Name of team's project - WordCLoud
    t8.columns[126]: "MO",
    t8.columns[127]: "Event Sub-Type",
    t8.columns[128]: "Event Type",
    t8.columns[129]: "Partner",
    t8.columns[145]: "STEAM Seasons",
    t8.columns[162]: "No. of Meet-ups",
    t8.columns[174]: "Stemettes Opportunities", #Internal Opps 
    t8.columns[175]: "Non-Stemettes Opportunities", #External Opps
    t8.columns[179]: "Successful Responses", #External Opps
    t8.columns[181]: "Planning to stay in touch with Mentor", #External Opps
    t8.columns[184]: "Goals Progress",
    t8.columns[189]: "Recommend to other Young People",
    t8.columns[210]: "Skills Progress",
    t8.columns[224]: "Enjoyed Meal Plans",
    t8.columns[227]: "Take Part in another",
    t8.columns[232]: "(Opportunities within/outside or Stemettes)",
    t8.columns[248]: "Record_ID (Unique)",
    t8.columns[249]: "First Time Attendee",
}

# (4.3b) Apply the Main Table renaming to the Feedback table
t8 = t8.rename(columns=t8_column_mapping)

# (4.3c) Get the list of new t8 column names (values from your mapping)
renamed_t8_columns = list(t8_column_mapping.values())

print("Renaming:", t8_column_mapping)

Renaming: {'Record_ID (Feedback)': 'Record_ID (Feedback)', 'Programmes x FB': 'Event', 'Start Date': 'Start Date', 'Record_ID (Onboarding)': 'Record_ID (Onb)', 'FB x Stage': 't8 Stage', 'Stream': 'Stream_1', 'Eligibility x FB': 'Age Group', 'Stream (Based on Age Today)': 'Stream_2', 'UK Region (from *Contacts x Eligibility) (from Eligibility x FB)': 'Region', 'Experience /10': 'Experience /10', 'Content /10': 'Content /10', 'Inn & PS improved?': 'Inn & PS Imp', 'Pre-event STEM Vs. STEAM CONNECTION': 'Pre-event STEM Vs. STEAM Conn', 'STEAM Separate?': 'STEAM Separate?', 'Post-event STEM Vs. STEAM CONNECTION': 'Post-event STEM Vs. STEAM CONNECTION', 'Connection Improvement?': 'Conn Imp', 'Perception_Improved?': 'Perception Imp', 'STEAM Perception_After': 'STEAM Perception_After', 'STEAM Awareness_After': 'STEAM Awareness_After', 'STEAM Network_After': 'STEAM Network_After', 'STEAM Confidence_After': 'STEAM Confidence_After', 'STEAM Knowledge_After': 'STEAM Knowledge_After', 'Mental Well-

In [44]:
# (4.4) Build a DataFrame showing t8 old name vs new name
t8_rename_comparison = pd.DataFrame({
    "t8 Original": list(t8_column_mapping.keys()),
    "t8 Renamed": list(t8_column_mapping.values())
})

#Show the table directly
t8_rename_comparison

from IPython.display import display, HTML

#Nicely formatted HTML version
display(HTML(f"""
    <div style="flex: 1;">
        <h3>Feedback Data (t8) Column Mapping</h3>
        {t8_rename_comparison.to_html(index=False)}
    </div>
"""))

t8 Original,t8 Renamed
Record_ID (Feedback),Record_ID (Feedback)
Programmes x FB,Event
Start Date,Start Date
Record_ID (Onboarding),Record_ID (Onb)
FB x Stage,t8 Stage
Stream,Stream_1
Eligibility x FB,Age Group
Stream (Based on Age Today),Stream_2
UK Region (from *Contacts x Eligibility) (from Eligibility x FB),Region
Experience /10,Experience /10


In [45]:
# (4.5a) Use only the renamed columns from each dataframe
Fb = t8[renamed_t8_columns]

# (4.5b) Concatenate the Renamed DfE Tables into 1 - #No need to concat
#Fb = pd.concat([t8_renamed], axis=0)

# (4.5c) #Validate transformation process- Check the before/after shape of the concatenated Dataframe - #No need to validate
#print("Rows before:", len(t8_renamed), "| Rows after:", len(Fb))
print(Fb.shape)

#Check that all the values in the Record_ID column are Unique (no duplicates) - #No need to check for duplicates
#print(Fb['Record_ID'].value_counts().max())

#Check that all the values in the Record_ID (Unique) column are not Unique (contain duplicates) - #No need to check for duplicate values
#print(Fb['Record_ID (Unique)'].value_counts().max())

#Remove Index and use Record_ID as Primary field
#Fb = Fb.set_index('Record_ID')

#Optional Preview of the first 3 rows only
Fb.head(3)

(714, 58)


,Record_ID (Feedback),Event,Start Date,Record_ID (Onb),t8 Stage,Stream_1,Age Group,Age Group,Stream_2,Region,Experience /10,Content /10,Inn & PS Imp,Pre-event STEM Vs. STEAM Conn,STEAM Separate?,Post-event STEM Vs. STEAM CONNECTION,Conn Imp,Perception Imp,STEAM Perception_After,STEAM Awareness_After,STEAM Network_After,STEAM Confidence_After,STEAM Knowledge_After,STEAM Well-being_After,Awareness Imp,Network Imp,Confidence Imp,Knowledge Imp,Well-being Imp,Self-Confidence Imp,Creativity Imp,Pre-event Intentions,Pre-event Intentions,Pre STEAM,Pre non-STEAM,Pre Uncertain,Post-event Intentions,STEAM (All),Likelihood (Both),Team Name,MO,Event Sub-Type,Event Type,Partner,STEAM Seasons,No. of Meet-ups,Stemettes Opportunities,Non-Stemettes Opportunities,Successful Responses,Planning to stay in touch with Mentor,Goals Progress,Recommend to other Young People,Skills Progress,Enjoyed Meal Plans,Take Part in another,(Opportunities within/outside or Stemettes),Record_ID (Unique),First Time Attendee
0,recmjPEoK9ZFvhVyg,Lenovo SLA,29/1/2025,recxGwTGTzDZJArwF,NaN,Age Not Disclosed,NaN,NaN,NaN,NaN,9.0,8.0,NaN,Somewhat SEPARATE,❌,I think STEM and the Arts are MORE CONNECTED,✅,✅,Improved a little,Improved a lot,Improved a little,Improved a little,Improved a little,Improved a little,✅,✅,✅,✅,✅,✅,✅,STEAM subject/career,STEAM Before,✅,NaN,NaN,STEAM Before | Non-STEAM After,NaN,✅,NaN,Physical,Stemettes Leadership Academy (SLA),EVENTS,Lenovo,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✅,NaN,NaN,GHOST/Not Yet Att
1,recGUv8kbuC9kkbLQ,Lenovo SLA,29/1/2025,recxGwTGTzDZJArwF,NaN,Age Not Disclosed,NaN,NaN,NaN,NaN,10.0,10.0,NaN,Somewhat SEPARATE,❌,I think STEM and the Arts are MORE CONNECTED,✅,✅,Improved a lot,Improved a lot,Improved a lot,Improved a lot,Improved a lot,Improved a lot,✅,✅,✅,✅,✅,✅,✅,I was not sure what I wanted to study/pursue,Uncertain Before,NaN,NaN,✅,Uncertain Before | Uncertain After,NaN,✅,NaN,Physical,Stemettes Leadership Academy (SLA),EVENTS,Lenovo,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✅,NaN,NaN,GHOST/Not Yet Att
2,rec6qd7chZKjBb4zE,Lenovo SLA,29/1/2025,recxGwTGTzDZJArwF,NaN,Age Not Disclosed,NaN,NaN,NaN,NaN,10.0,8.0,NaN,Somewhat SEPARATE,❌,I think STEM and the Arts are MORE CONNECTED,✅,✅,Improved a lot,Improved a little,Improved a lot,Improved a lot,Improved a lot,Improved a lot,✅,✅,✅,✅,✅,✅,✅,I was not sure what I wanted to study/pursue,Uncertain Before,NaN,NaN,✅,Uncertain Before | STEAM After,✅,NaN,NaN,Physical,Stemettes Leadership Academy (SLA),EVENTS,Lenovo,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✅,NaN,NaN,GHOST/Not Yet Att


In [46]:
# (4.6) #No need to drop or delete any columns
#Make a copy of the newly concatenated df for easy referencing to the original (WITHOUT ANY columns that should remain anonymous i.e. AFTER we dropped the _ column)
#Fb_anon = Fb.drop(columns=["DELETE"])

#Check the new shape of the concatenated and anonymised Dataframe
#print(Fb_anon.shape)

#Preview the concatenated and anonymous dataframe
#Fb_anon.head(3)

In [47]:
# (4.7a) Apply DataType Transformations to 'Anonymous' Dataset
#Ensure x, y and z values are Integers
#Fb.loc[:, 'x'] = pd.to_numeric(Fb['x'], errors='coerce').astype('Int64')

#Convert date fields to ISO 8:
Fb['Start Date'] = pd.to_datetime(Fb['Start Date'], errors='coerce', dayfirst=True)

#Make a copy after applying transformations to concatenated dataset, before creating any slices of the original data
Fb_tf = Fb.copy()

#Check the new shape of the transformed Dataframe
print(Fb_tf.shape)

#Preview the result
Fb_tf.head(3)

(714, 58)


,Record_ID (Feedback),Event,Start Date,Record_ID (Onb),t8 Stage,Stream_1,Age Group,Age Group,Stream_2,Region,Experience /10,Content /10,Inn & PS Imp,Pre-event STEM Vs. STEAM Conn,STEAM Separate?,Post-event STEM Vs. STEAM CONNECTION,Conn Imp,Perception Imp,STEAM Perception_After,STEAM Awareness_After,STEAM Network_After,STEAM Confidence_After,STEAM Knowledge_After,STEAM Well-being_After,Awareness Imp,Network Imp,Confidence Imp,Knowledge Imp,Well-being Imp,Self-Confidence Imp,Creativity Imp,Pre-event Intentions,Pre-event Intentions,Pre STEAM,Pre non-STEAM,Pre Uncertain,Post-event Intentions,STEAM (All),Likelihood (Both),Team Name,MO,Event Sub-Type,Event Type,Partner,STEAM Seasons,No. of Meet-ups,Stemettes Opportunities,Non-Stemettes Opportunities,Successful Responses,Planning to stay in touch with Mentor,Goals Progress,Recommend to other Young People,Skills Progress,Enjoyed Meal Plans,Take Part in another,(Opportunities within/outside or Stemettes),Record_ID (Unique),First Time Attendee
0,recmjPEoK9ZFvhVyg,Lenovo SLA,2025-01-29,recxGwTGTzDZJArwF,NaN,Age Not Disclosed,NaN,NaN,NaN,NaN,9.0,8.0,NaN,Somewhat SEPARATE,❌,I think STEM and the Arts are MORE CONNECTED,✅,✅,Improved a little,Improved a lot,Improved a little,Improved a little,Improved a little,Improved a little,✅,✅,✅,✅,✅,✅,✅,STEAM subject/career,STEAM Before,✅,NaN,NaN,STEAM Before | Non-STEAM After,NaN,✅,NaN,Physical,Stemettes Leadership Academy (SLA),EVENTS,Lenovo,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✅,NaN,NaN,GHOST/Not Yet Att
1,recGUv8kbuC9kkbLQ,Lenovo SLA,2025-01-29,recxGwTGTzDZJArwF,NaN,Age Not Disclosed,NaN,NaN,NaN,NaN,10.0,10.0,NaN,Somewhat SEPARATE,❌,I think STEM and the Arts are MORE CONNECTED,✅,✅,Improved a lot,Improved a lot,Improved a lot,Improved a lot,Improved a lot,Improved a lot,✅,✅,✅,✅,✅,✅,✅,I was not sure what I wanted to study/pursue,Uncertain Before,NaN,NaN,✅,Uncertain Before | Uncertain After,NaN,✅,NaN,Physical,Stemettes Leadership Academy (SLA),EVENTS,Lenovo,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✅,NaN,NaN,GHOST/Not Yet Att
2,rec6qd7chZKjBb4zE,Lenovo SLA,2025-01-29,recxGwTGTzDZJArwF,NaN,Age Not Disclosed,NaN,NaN,NaN,NaN,10.0,8.0,NaN,Somewhat SEPARATE,❌,I think STEM and the Arts are MORE CONNECTED,✅,✅,Improved a lot,Improved a little,Improved a lot,Improved a lot,Improved a lot,Improved a lot,✅,✅,✅,✅,✅,✅,✅,I was not sure what I wanted to study/pursue,Uncertain Before,NaN,NaN,✅,Uncertain Before | STEAM After,✅,NaN,NaN,Physical,Stemettes Leadership Academy (SLA),EVENTS,Lenovo,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✅,NaN,NaN,GHOST/Not Yet Att


In [48]:
# (4.7b) Map Likert-style feedback to numeric uplift on the transformed DF
Fb_map = Fb_tf.copy()

likert_map = {
    'Improved a lot': 20,
    'Improved a little': 10,
    'Not changed': 0,
    'Become worse': -10
}

# Columns to map
likert_cols = [
    'STEAM Perception_After', 'STEAM Awareness_After', 'STEAM Network_After',
    'STEAM Confidence_After', 'STEAM Knowledge_After', 'STEAM Well-being_After'
]

# Optional: preserve originals
for c in likert_cols:
    orig = f"{c} (orig)"
    if orig not in Fb_map.columns:
        Fb_map[orig] = Fb_map[c]

# Normalize text, then map → numeric Int8
for c in likert_cols:
    Fb_map[c] = (
        Fb_map[c]
        .astype('string')
        .str.strip()
        .str.replace(r'\s+', ' ', regex=True)
        .map(likert_map)
        .astype('Int8')  # keeps NA if unknown
    )

# Continue pipeline using Fb_map instead of Fb_tf
Fb_new = Fb_map.copy()  # (4.8) Add any new columns for calculations…

In [49]:
# (4.8) Add any new columns for calculations
#Make a copy before adding a new column to the transformed dataframe
Fb_new = Fb_tf.copy()

#Create a new column called x that does y for each row
#Fb_new['x'] = Fb_new[['y']].bfill(axis=1).iloc[:, 0]

#Check the new shape of the Dataframe
print(Fb_new.shape)

#Preview the result
Fb_new.head()

(714, 58)


,Record_ID (Feedback),Event,Start Date,Record_ID (Onb),t8 Stage,Stream_1,Age Group,Age Group,Stream_2,Region,Experience /10,Content /10,Inn & PS Imp,Pre-event STEM Vs. STEAM Conn,STEAM Separate?,Post-event STEM Vs. STEAM CONNECTION,Conn Imp,Perception Imp,STEAM Perception_After,STEAM Awareness_After,STEAM Network_After,STEAM Confidence_After,STEAM Knowledge_After,STEAM Well-being_After,Awareness Imp,Network Imp,Confidence Imp,Knowledge Imp,Well-being Imp,Self-Confidence Imp,Creativity Imp,Pre-event Intentions,Pre-event Intentions,Pre STEAM,Pre non-STEAM,Pre Uncertain,Post-event Intentions,STEAM (All),Likelihood (Both),Team Name,MO,Event Sub-Type,Event Type,Partner,STEAM Seasons,No. of Meet-ups,Stemettes Opportunities,Non-Stemettes Opportunities,Successful Responses,Planning to stay in touch with Mentor,Goals Progress,Recommend to other Young People,Skills Progress,Enjoyed Meal Plans,Take Part in another,(Opportunities within/outside or Stemettes),Record_ID (Unique),First Time Attendee
0,recmjPEoK9ZFvhVyg,Lenovo SLA,2025-01-29,recxGwTGTzDZJArwF,NaN,Age Not Disclosed,NaN,NaN,NaN,NaN,9.0,8.0,NaN,Somewhat SEPARATE,❌,I think STEM and the Arts are MORE CONNECTED,✅,✅,Improved a little,Improved a lot,Improved a little,Improved a little,Improved a little,Improved a little,✅,✅,✅,✅,✅,✅,✅,STEAM subject/career,STEAM Before,✅,NaN,NaN,STEAM Before | Non-STEAM After,NaN,✅,NaN,Physical,Stemettes Leadership Academy (SLA),EVENTS,Lenovo,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✅,NaN,NaN,GHOST/Not Yet Att
1,recGUv8kbuC9kkbLQ,Lenovo SLA,2025-01-29,recxGwTGTzDZJArwF,NaN,Age Not Disclosed,NaN,NaN,NaN,NaN,10.0,10.0,NaN,Somewhat SEPARATE,❌,I think STEM and the Arts are MORE CONNECTED,✅,✅,Improved a lot,Improved a lot,Improved a lot,Improved a lot,Improved a lot,Improved a lot,✅,✅,✅,✅,✅,✅,✅,I was not sure what I wanted to study/pursue,Uncertain Before,NaN,NaN,✅,Uncertain Before | Uncertain After,NaN,✅,NaN,Physical,Stemettes Leadership Academy (SLA),EVENTS,Lenovo,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✅,NaN,NaN,GHOST/Not Yet Att
2,rec6qd7chZKjBb4zE,Lenovo SLA,2025-01-29,recxGwTGTzDZJArwF,NaN,Age Not Disclosed,NaN,NaN,NaN,NaN,10.0,8.0,NaN,Somewhat SEPARATE,❌,I think STEM and the Arts are MORE CONNECTED,✅,✅,Improved a lot,Improved a little,Improved a lot,Improved a lot,Improved a lot,Improved a lot,✅,✅,✅,✅,✅,✅,✅,I was not sure what I wanted to study/pursue,Uncertain Before,NaN,NaN,✅,Uncertain Before | STEAM After,✅,NaN,NaN,Physical,Stemettes Leadership Academy (SLA),EVENTS,Lenovo,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✅,NaN,NaN,GHOST/Not Yet Att
3,recZIEDOBnSmg6LZx,Lenovo SLA,2025-01-29,recxGwTGTzDZJArwF,NaN,Age Not Disclosed,NaN,NaN,NaN,NaN,8.0,8.0,NaN,Somewhat CONNECTED,NaN,I still see STEAM the SAME way as I did before,NaN,✅,Improved a lot,Improved a lot,Improved a lot,Improved a lot,Improved a lot,Improved a lot,✅,✅,✅,✅,✅,✅,✅,STEAM subject/career,STEAM Before,✅,NaN,NaN,STEAM Before | STEAM Still,✅,✅,NaN,Physical,Stemettes Leadership Academy (SLA),EVENTS,Lenovo,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✅,NaN,NaN,GHOST/Not Yet Att
4,recKf4xWyTGTdju0w,Lenovo SLA,2025-01-29,recxGwTGTzDZJArwF,NaN,Age Not Disclosed,NaN,NaN,NaN,NaN,9.0,8.0,NaN,Somewhat SEPARATE,❌,I think STEM and the Arts are MORE CONNECTED,✅,✅,Improved a little,Improved a lot,Improved a little,Improved a little,Improved a lot,Not changed,✅,✅,✅,✅,NaN,✅,✅,STEAM subject/career,STEAM Before,✅,NaN,NaN,STEAM Before | STEAM Still,✅,✅,NaN,Physical,Stemettes Leadership Academy (SLA),EVENTS,Lenovo,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,✅,NaN,NaN,GHOST/Not Yet Att


In [50]:
# Count non-NaN values for each column in Fb
Fb_non_nan_counts = Fb.notna().sum()

print("Non-NaN counts per column:")
print(Fb_non_nan_counts)

# If you want the total across the whole dataframe:
Fb_total_non_nan = Fb.notna().sum().sum()
print(f"\nTotal non-NaN values in Fb: {Fb_total_non_nan}")

#Count Non-NaN Values in Record_ID (Unique) column from t8_renamed table
Fb_non_nan_record_ids = Fb["Record_ID (Unique)"].notna().sum()
print(f"Fb Non-NaN Record_ID (Unique) values: {Fb_non_nan_record_ids}")

Non-NaN counts per column:
Record_ID (Feedback)                           714
Event                                          714
Start Date                                     714
Record_ID (Onb)                                704
t8 Stage                                        91
Stream_1                                       714
Age Group                                      639
Age Group                                      122
Stream_2                                       637
Region                                         339
Experience /10                                 714
Content /10                                    651
Inn & PS Imp                                   129
Pre-event STEM Vs. STEAM Conn                  638
STEAM Separate?                                214
Post-event STEM Vs. STEAM CONNECTION           570
Conn Imp                                       366
Perception Imp                                 568
STEAM Perception_After                         538
STEA

In [51]:
# (4.9) Misc
Fb_matched = Fb_new[Fb_new['Record_ID (Onb)'].notna() & Fb_new['Record_ID (Onb)'].isin(Att_v3['Record_ID (Unique)'].dropna())]

#Convert t8_matched list to dataframe
Fb_matched_df = pd.DataFrame(Fb_matched)

#Check number of records from Feedback that appear in the consolidated UniqPp table
print("Unique Feedback Shape (pre-concat):", Fb_matched_df.shape)
Fb_matched_df.head()

#Drop previous "_merge" column
for df in (Att_v3, Fb_matched_df):
    df.drop(columns=["_merge"], errors="ignore", inplace=True)

#Cardinality = Main Table:DfE table (Many:One), duplicates in Table 1, no duplicates in Table 2 - Keep all data in the main, only match data in right
UniqPp_Fb = Att_v3.merge(Fb_matched_df, how="left", on="Record_ID (Unique)", indicator=True)

#Check New Merged shape with 15/16 selected columns added from the DfE Index
print("Unique Feedback Shape (post-concat):", UniqPp_Fb.shape)

#Alternative joining code
#df1 = pd.DataFrame(Att_v3)
#df2 = pd.DataFrame(Fb_matched_df)
#UniqPp_Fb = df1.merge(df2, how='left', on="Record_ID (Onb)", indicator=True)
#UniqPp_Fb

#Count how many rows matched vs did not match
uniquefeedback_match_counts = UniqPp_Fb["_merge"].value_counts()

#Calculate match percentage
uniquefeedback_match_rate = (uniquefeedback_match_counts.get("both", 0) / len(UniqPp_Fb)) * 100

print("Unique Feedback Match counts:")
print(uniquefeedback_match_counts)
print(f"\nMatch rate: {uniquefeedback_match_rate:.2f}% of unique participant records found in the Feedback Table")

UniqPp_Fb.head()

Unique Feedback Shape (pre-concat): (0, 58)
Unique Feedback Shape (post-concat): (11425, 132)
Unique Feedback Match counts:
_merge
left_only     11425
right_only        0
both              0
Name: count, dtype: int64

Match rate: 0.00% of unique participant records found in the Feedback Table


,Record_ID,Event_x,STEAM Season,Partner_x,Event Sub-Type_x,Year,Year-Quarter-Month,Type,Quantity,URN_1,Region_x,LSOA (%),State-funded (%),FSM (%),Age Today_x,Stream,Age on Day of,P/G Attend Uni,P/G Work in STEAM,Pronouns,Ethnic Background,Physical / Virtual,UK-Based,Ethnic Background (Grouped),Total Students Engaged,How Did You Hear,Where Did You Hear,QQ,Non-UK based country,Record_ID (Onb)_x,URN_2,URN_3,URN_4,Onboarded Date/Time,First Time Attendee? (Y/N)_x,Record_ID (Unique),Sign-up Date/Time,STEAM Tool,Record_ID (Fb),# Fb,Fb sub date,Fb event date,Date/Time (1st Sign-up),URN,Local Authority Name,Establishment Name,Type of Establishment,Establishment Type Group,Statutory_Low_Age,Statutory_High_Age,No. of Pupils,Percentage FSM,Street,Locality,Town,Postcode,Ofsted Rating,LSOA (code),FSM,# Sign-ups,# Onb,Att (Y/N),# Att,Att 1+,First Time Attendee? (Y/N)_y,Points,# Feeback,Where_heard_about,Age Today_y,SSphere Sign-up Date/Time,Email Opt-in (Y/N),In SSoc?,In SFJB?,In SFYB?,Record_ID (Feedback),Event_y,Start Date,Record_ID (Onb)_y,t8 Stage,Stream_1,Age Group,Age Group,Stream_2,Region_y,Experience /10,Content /10,Inn & PS Imp,Pre-event STEM Vs. STEAM Conn,STEAM Separate?,Post-event STEM Vs. STEAM CONNECTION,Conn Imp,Perception Imp,STEAM Perception_After,STEAM Awareness_After,STEAM Network_After,STEAM Confidence_After,STEAM Knowledge_After,STEAM Well-being_After,Awareness Imp,Network Imp,Confidence Imp,Knowledge Imp,Well-being Imp,Self-Confidence Imp,Creativity Imp,Pre-event Intentions,Pre-event Intentions,Pre STEAM,Pre non-STEAM,Pre Uncertain,Post-event Intentions,STEAM (All),Likelihood (Both),Team Name,MO,Event Sub-Type_y,Event Type,Partner_y,STEAM Seasons,No. of Meet-ups,Stemettes Opportunities,Non-Stemettes Opportunities,Successful Responses,Planning to stay in touch with Mentor,Goals Progress,Recommend to other Young People,Skills Progress,Enjoyed Meal Plans,Take Part in another,(Opportunities within/outside or Stemettes),First Time Attendee,_merge
0,recUrZnORsIVbVFwr,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,Region TBC,LSOA TBC,1.0,100%,17.0,Academy,16.0,No,No,She/ Her,Black or Black British - African,I will be joining in-person,Yes,Black or Black British,NaN,Through School / College / Uni,NaN,NaN,NaN,recvL2l853VHDlFzP,NaN,NaN,<NA>,11/7/2025 9:35am,EXISTING/Repeat Attendee,recMWiC7pw1ZO0hfe,10/7/2025 4:26pm,NaN,recwapMx1jjXPUZlp,1,19/7/2025,19/7/2025,10/7/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,2.0,2.0,2 checked out of 2,2.0,1.0,EXISTING/Repeat Attendee,"10, 40",2.0,NaN,17,2025-07-10,0 checked out of 1,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,recqe2XUQ0eSk0CTf,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,London,4,1.0,100%,17.0,Academy,17.0,No,Yes,She/ Her,Mixed - White and Black African,I will be joining in-person,Yes,Mixed,NaN,Stemette Newsletter,NaN,NaN,NaN,rechqToGPUTdlChai,NaN,NaN,<NA>,12/5/2025 2:37pm,NEW/First-time Attendee,recrPMUI4ZCTOkgop,30/4/2025 9:01pm,NaN,NaN,0,NaN,NaN,30/4/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,1.0,2.0,1 checked out of 2,1.0,1.0,NEW/First-time Attendee,10,1.0,NaN,17,2025-04-30,0 checked out of 1,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,recUdaEr0T20VEL5N,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Ev

In [52]:
# (4.10) Make a copy before making any modifications to the concatenated and merged data frame
Att_new4 = UniqPp_Fb.copy()

#Convert joined list to dataframe
Att_v4 = pd.DataFrame(Att_new4)
Att_v4.head()

#Check New Merged shape with 15/16 selected columns added from the DfE Index
print("Shape:", Att_v4.shape)

#Generate Summary Stats for Numerical Data
##Descriptive Stats for different time-periods, streams, regions, types... maybe boss- repeat this process for sign-ups
Att_v4.describe()

Shape: (11425, 132)


,Year,Quantity,State-funded (%),Age Today_x,Age on Day of,Total Students Engaged,# Fb,Statutory_Low_Age,Statutory_High_Age,No. of Pupils,Percentage FSM,FSM,# Sign-ups,# Onb,# Att,Att 1+,# Feeback,Age Today_y,SSphere Sign-up Date/Time,Start Date,Experience /10,Content /10,Team Name,No. of Meet-ups
count,11425.000000,11425.0,9957.000000,1237.000000,2432.000000,3006.000000,11425.000000,6484.0,6484.0,6484.0,6484.000000,6484.0,1236.000000,1236.000000,1236.000000,1236.000000,1236.000000,1233.0,1236,0,0.0,0.0,0.0,0.0
mean,2023.595536,127.137418,0.904991,15.413096,13.112664,18.536593,0.073961,8.175663,15.462523,800.192474,23.325200,152.896977,4.127832,3.652104,2.766990,0.991909,2.322006,15.30738,2024-10-28 03:51:50.679611648,NaT,NaN,NaN,NaN,NaN
min,2023.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,7.0,0.0,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,2023-03-24 00:00:00,NaT,NaN,NaN,NaN,NaN
25%,2023.000000,10.0,1.000000,12.000000,11.000000,10.000000,0.000000,4.0,11.0,393.0,10.000000,33.0,1.000000,1.000000,1.000000,1.000000,0.000000,12.0,2024-05-16 00:00:00,NaT,NaN,NaN,NaN,NaN
50%,2024.000000,10.0,1.000000,16.000000,12.000000,10.000000,0.000000,11.0,16.0,704.0,24.500000,105.0,2.000000,2.000000,1.000000,1.000000,1.000000,15.0,2024-10-30 12:00:00,NaT,NaN,NaN,NaN,NaN
75%,2024.000000,200.0,1.000000,18.000000,15.000000,28.000000,0.000000,11.0,18.0,1163.0,33.100000,277.0,5.000000,4.000000,3.000000,1.000000,3.000000,18.0,2025-04-20 06:00:00,NaT,NaN,NaN,NaN,NaN
max,2025.000000,650.0,1.000000,63.000000,63.000000,90.000000,5.000000,16.0,25.0,2990.0,77.600000,660.0,28.000000,26.000000,20.000000,1.000000,25.000000,63.0,2025-08-28 00:00:00,NaT,NaN,NaN,NaN,NaN
std,0.605036,198.288198,0.293241,5.092979,3.841423,16.801112,0.330322,3.757063,3.178185,497.437269,15.747513,132.055285,5.311038,4.637674,3.363882,0.089620,3.840530,5.135541,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
#Check how many values in the Record_ID column are Unique (no duplicates)
print("Total Fb Records:", Att_v4['Record_ID'].nunique()) #Print no. of unique entries

#Check how many values in the Record_ID column are NOT Unique (contain duplicates)
print("Duplicate Total Fb Records:", Att_v4['Record_ID'].value_counts().max())

#Check how many values in the Record_ID (Unique) column are Unique (no duplicates)
print("Unique Fb Records:", Att_v4['Record_ID (Unique)'].nunique()) #Print no. of unique entries

#Check how many values in the Record_ID (Unique) column are NOT Unique (contain duplicates)
print("Duplicate Unique Fb Records:", Att_v4['Record_ID (Unique)'].value_counts().max())

Total Fb Records: 11425
Duplicate Total Fb Records: 1
Unique Fb Records: 929
Duplicate Unique Fb Records: 14


In [54]:
# Make pandas display wider columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 200)

# Filter out nulls
non_null_att = Att_v4[Att_v4['Record_ID (Unique)'].notna()]

# Group by Record_ID (Unique), including singles
summary_all = (
    non_null_att.groupby('Record_ID (Unique)')
    .agg(
        Total_Count=('Record_ID', 'count'),
        Associated_Record_IDs=('Record_ID', lambda x: ", ".join(map(str, x)))
    )
    .reset_index()
)

# Calculate duplicates excluding first occurrence
summary_all['Duplicates_Excluding_First'] = summary_all['Total_Count'] - 1

# Sort by Duplicates_Excluding_First descending
summary_all_sorted = summary_all.sort_values(by='Duplicates_Excluding_First', ascending=False).reset_index(drop=True)

print("Summary view of all non-null Record_ID (Unique), including singles:")
summary_all_sorted.head(50)

Summary view of all non-null Record_ID (Unique), including singles:


,Record_ID (Unique),Total_Count,Associated_Record_IDs,Duplicates_Excluding_First
0,recCHtY6Lko1UKXDa,14,"recWj2cLSnFIQFgNn, rec54Q5W3dhAVGEvS, recpTVqiF1jUjMhys, recHvIB5M2yAf7hqj, recpEpECHwg7VjAF9, recjVYp7V85OKclaS, recK596rfwO33F8In, recQ3JJgiiryrGSpp, recinzrp7nvnUegcn, recJj2u9WYvgba3Cb, recPNIhs2bQCAbRfo, recVX7zJJrp2XThhM, recDNw0BfbLevkjtT, recM5eLdAokXH7EJu",13
1,recHXCgIG2d04ssNJ,12,"reckqSiOG2nYiCEAq, recB85Kt7vwkI2RzV, recNIT3rhtTcQ5LHT, recZd3Ro8V7qIeUL4, recM5BaFrGW8EyouM, reciermUItTbdF39v, rec5ciGotAEdhzhhA, recRm9pjIRiORg9k7, recc9NBRVUbzFCB1e, recrnGG5EoHRvjw7Y, recE9xBjRukft6lBn, recN8S0UZaa6JnvId",11
2,recH9QXD7vJ0CIi9r,9,"recdJCQoNmKyvLKcY, recYuXgyma7NAbkM1, recIfQFf6kGGEnpwd, recUDBmpqENDldJtG, rec6CCWRLuFrwpIH3, recPm6YYLY6vqnfSo, recSQhcWChWno7ju9, recpBKqUid2pFlAuJ, recELFi0FqtKVS0JI",8
3,recwbGGMf4KsuhAMQ,8,"recL3ogCPHyBDc5Wr, recE4pUW1LZkDfEv7, rec2tRCODnZeHnpWR, recdUCPmVmZ2jP9OQ, rec4V7Gu6FtrRTpKw, recIGB6VzVthavGm0, rec4Ih6Vp9INrbAEL, receTN4QzGMaQxsiN",7
4,recHpVO4pYFyYzr7l,7,"recKycGBFAI0Rv6Vb, rect64OLuiT7Z2XDV, recBPBwt55z8839yG, reczNMMMLKf3Fv8j4, recTUZhdTrfU1SnEm, reczpAX0qEhC5u0Qr, rec5VRR9kKvgusNlh",6
5,recAE7iIa6ivQ2Twc,7,"rec2AOohpMzyLHko0, recrODUUeV64DXnjY, recRf2Bow3YPqV6nV, recPwEideXSDb4NFE, rec05ilwMkvuYLK4e, recU89bbxhfxooybg, recaH1tXM2wW7qT0i",6
6,rec5jnEo1YpnkCNm1,6,"recRbLMAN5UNkbyTt, rec6BHU6L2KF0WkaB, recIogwofBcezvkZ6, reciFnqPdcvJOg2zm, rec3od6CGJgXJtz5o, recJqCts4M4p6ujBF",5
7,recTjTBqZLtc38SAk,6,"reciBvQyNiSgkljlX, rec6PAwgxYoLo04ss, reciGhXd6VbfAd9lE, rec2AV8oISaIgs9x0, recsnrcqXyM32EMYI, recLgnzBxaNR1efxQ",5
8,rec1tsvSA5WmSUg26,6,"recgyjLSRiJePOI9A, recwQXYLWcBqw0gzA, recdYVaxgu6kacIiW, recZNUsrzATq10hEI, recGxLBjulCnUVpgW, recg3jXwKeyXG1KAz",5
9,recCbb8CxHrUO8ldL,6,"recdgBSnokWWARSQA, recngZ9wcMVxVlBTN, recgcNjHEJeJQiOfs, recQUTfR5EOEXoBI5, recgTl66nStXZtR3u, reckdmOwCPb3Jfrag",5


## ✅ Shape & Variable Checking

For Att v1, 2, 3 & 4

In [55]:
print("Shape:", Att_v1.shape) #Added student rows in place of schools rows + added extra URN column
#Att_v1.head(5)

Shape: (11425, 44)


In [56]:
print("Shape:", Att_v2.shape) #Added 16 extra columns containing DfE Schools Data + removed merge field column
#Att_v2.head(5)

Shape: (11425, 59)


In [57]:
print("Shape:", Att_v3.shape) #Added 16 extra columns containing Demographic Data for matched Unique Participants + removed merge field column
#Att_v3.head(5)

Shape: (11425, 74)


In [58]:
print("Shape:", Att_v4.shape) #Added 51 extra columns containing Feedback Data for matched Unique Participants
Att_v4.head(5)

Shape: (11425, 132)


,Record_ID,Event_x,STEAM Season,Partner_x,Event Sub-Type_x,Year,Year-Quarter-Month,Type,Quantity,URN_1,Region_x,LSOA (%),State-funded (%),FSM (%),Age Today_x,Stream,Age on Day of,P/G Attend Uni,P/G Work in STEAM,Pronouns,Ethnic Background,Physical / Virtual,UK-Based,Ethnic Background (Grouped),Total Students Engaged,How Did You Hear,Where Did You Hear,QQ,Non-UK based country,Record_ID (Onb)_x,URN_2,URN_3,URN_4,Onboarded Date/Time,First Time Attendee? (Y/N)_x,Record_ID (Unique),Sign-up Date/Time,STEAM Tool,Record_ID (Fb),# Fb,Fb sub date,Fb event date,Date/Time (1st Sign-up),URN,Local Authority Name,Establishment Name,Type of Establishment,Establishment Type Group,Statutory_Low_Age,Statutory_High_Age,No. of Pupils,Percentage FSM,Street,Locality,Town,Postcode,Ofsted Rating,LSOA (code),FSM,# Sign-ups,# Onb,Att (Y/N),# Att,Att 1+,First Time Attendee? (Y/N)_y,Points,# Feeback,Where_heard_about,Age Today_y,SSphere Sign-up Date/Time,Email Opt-in (Y/N),In SSoc?,In SFJB?,In SFYB?,Record_ID (Feedback),Event_y,Start Date,Record_ID (Onb)_y,t8 Stage,Stream_1,Age Group,Age Group,Stream_2,Region_y,Experience /10,Content /10,Inn & PS Imp,Pre-event STEM Vs. STEAM Conn,STEAM Separate?,Post-event STEM Vs. STEAM CONNECTION,Conn Imp,Perception Imp,STEAM Perception_After,STEAM Awareness_After,STEAM Network_After,STEAM Confidence_After,STEAM Knowledge_After,STEAM Well-being_After,Awareness Imp,Network Imp,Confidence Imp,Knowledge Imp,Well-being Imp,Self-Confidence Imp,Creativity Imp,Pre-event Intentions,Pre-event Intentions,Pre STEAM,Pre non-STEAM,Pre Uncertain,Post-event Intentions,STEAM (All),Likelihood (Both),Team Name,MO,Event Sub-Type_y,Event Type,Partner_y,STEAM Seasons,No. of Meet-ups,Stemettes Opportunities,Non-Stemettes Opportunities,Successful Responses,Planning to stay in touch with Mentor,Goals Progress,Recommend to other Young People,Skills Progress,Enjoyed Meal Plans,Take Part in another,(Opportunities within/outside or Stemettes),First Time Attendee,_merge
0,recUrZnORsIVbVFwr,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,Region TBC,LSOA TBC,1.0,100%,17.0,Academy,16.0,No,No,She/ Her,Black or Black British - African,I will be joining in-person,Yes,Black or Black British,NaN,Through School / College / Uni,NaN,NaN,NaN,recvL2l853VHDlFzP,NaN,NaN,<NA>,11/7/2025 9:35am,EXISTING/Repeat Attendee,recMWiC7pw1ZO0hfe,10/7/2025 4:26pm,NaN,recwapMx1jjXPUZlp,1,19/7/2025,19/7/2025,10/7/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,2.0,2.0,2 checked out of 2,2.0,1.0,EXISTING/Repeat Attendee,"10, 40",2.0,NaN,17,2025-07-10,0 checked out of 1,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,recqe2XUQ0eSk0CTf,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,London,4,1.0,100%,17.0,Academy,17.0,No,Yes,She/ Her,Mixed - White and Black African,I will be joining in-person,Yes,Mixed,NaN,Stemette Newsletter,NaN,NaN,NaN,rechqToGPUTdlChai,NaN,NaN,<NA>,12/5/2025 2:37pm,NEW/First-time Attendee,recrPMUI4ZCTOkgop,30/4/2025 9:01pm,NaN,NaN,0,NaN,NaN,30/4/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,1.0,2.0,1 checked out of 2,1.0,1.0,NEW/First-time Attendee,10,1.0,NaN,17,2025-04-30,0 checked out of 1,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,recUdaEr0T20VEL5N,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Ev

In [59]:
#Build DataFrames with only renamed column names
Att_v1_list = pd.Series(list(t1_column_mapping.values()), name="Attendance Data (T1)")
Att_v2_list = pd.Series(list(t6_column_mapping.values()), name="DfE Schools Data (T6)")
Att_v3_list = pd.Series(list(t7_column_mapping.values()), name="Unique Participant Demographic Data (T7)")
Att_v4_list = pd.Series(list(t8_column_mapping.values()), name="Feedback Data (T8)")

#Combine side by side
renamed_only = pd.concat([Att_v1_list, Att_v2_list, Att_v3_list, Att_v4_list], axis=1)

#Show the renamed-only DataFrame
renamed_only

#Optional: nice HTML display
from IPython.display import display, HTML

display(HTML(f"""
    <div style="margin: 8px 0;">
        <h3 style="margin:0 0 8px 0;">Variables for Exploratory Analysis from Attendance Data, DfE Schools Data, Particpant Demographic Data and Feedback Data:</h3>
        {renamed_only.fillna("").to_html(index=False)}
    </div>
"""))

Attendance Data (T1),DfE Schools Data (T6),Unique Participant Demographic Data (T7),Feedback Data (T8)
Record_ID,URN,Record_ID (Unique),Record_ID (Feedback)
Event,Local Authority Name,# Sign-ups,Event
STEAM Season,Establishment Name,# Onb,Start Date
Partner,Type of Establishment,Att (Y/N),Record_ID (Onb)
Event Sub-Type,Establishment Type Group,# Att,t8 Stage
Year,Statutory_Low_Age,Att 1+,Stream_1
Year-Quarter-Month,Statutory_High_Age,First Time Attendee? (Y/N),Age Group
Type,No. of Pupils,Points,Stream_2
Quantity,Percentage FSM,# Feeback,Region
URN_1,Street,Where_heard_about,Experience /10


# **✨The Clean Aggregated Dataset:**
## Shape: 11425 Rows x 132 Columns

This dataset (Att_new4) contains the Attendance Data of Young People & Students and links with Demographics and Feedback.

In [60]:
print(Att_new4.shape) #Error-checking
Att_new4.head(5) #Error-checking

(11425, 132)


,Record_ID,Event_x,STEAM Season,Partner_x,Event Sub-Type_x,Year,Year-Quarter-Month,Type,Quantity,URN_1,Region_x,LSOA (%),State-funded (%),FSM (%),Age Today_x,Stream,Age on Day of,P/G Attend Uni,P/G Work in STEAM,Pronouns,Ethnic Background,Physical / Virtual,UK-Based,Ethnic Background (Grouped),Total Students Engaged,How Did You Hear,Where Did You Hear,QQ,Non-UK based country,Record_ID (Onb)_x,URN_2,URN_3,URN_4,Onboarded Date/Time,First Time Attendee? (Y/N)_x,Record_ID (Unique),Sign-up Date/Time,STEAM Tool,Record_ID (Fb),# Fb,Fb sub date,Fb event date,Date/Time (1st Sign-up),URN,Local Authority Name,Establishment Name,Type of Establishment,Establishment Type Group,Statutory_Low_Age,Statutory_High_Age,No. of Pupils,Percentage FSM,Street,Locality,Town,Postcode,Ofsted Rating,LSOA (code),FSM,# Sign-ups,# Onb,Att (Y/N),# Att,Att 1+,First Time Attendee? (Y/N)_y,Points,# Feeback,Where_heard_about,Age Today_y,SSphere Sign-up Date/Time,Email Opt-in (Y/N),In SSoc?,In SFJB?,In SFYB?,Record_ID (Feedback),Event_y,Start Date,Record_ID (Onb)_y,t8 Stage,Stream_1,Age Group,Age Group,Stream_2,Region_y,Experience /10,Content /10,Inn & PS Imp,Pre-event STEM Vs. STEAM Conn,STEAM Separate?,Post-event STEM Vs. STEAM CONNECTION,Conn Imp,Perception Imp,STEAM Perception_After,STEAM Awareness_After,STEAM Network_After,STEAM Confidence_After,STEAM Knowledge_After,STEAM Well-being_After,Awareness Imp,Network Imp,Confidence Imp,Knowledge Imp,Well-being Imp,Self-Confidence Imp,Creativity Imp,Pre-event Intentions,Pre-event Intentions,Pre STEAM,Pre non-STEAM,Pre Uncertain,Post-event Intentions,STEAM (All),Likelihood (Both),Team Name,MO,Event Sub-Type_y,Event Type,Partner_y,STEAM Seasons,No. of Meet-ups,Stemettes Opportunities,Non-Stemettes Opportunities,Successful Responses,Planning to stay in touch with Mentor,Goals Progress,Recommend to other Young People,Skills Progress,Enjoyed Meal Plans,Take Part in another,(Opportunities within/outside or Stemettes),First Time Attendee,_merge
0,recUrZnORsIVbVFwr,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,Region TBC,LSOA TBC,1.0,100%,17.0,Academy,16.0,No,No,She/ Her,Black or Black British - African,I will be joining in-person,Yes,Black or Black British,NaN,Through School / College / Uni,NaN,NaN,NaN,recvL2l853VHDlFzP,NaN,NaN,<NA>,11/7/2025 9:35am,EXISTING/Repeat Attendee,recMWiC7pw1ZO0hfe,10/7/2025 4:26pm,NaN,recwapMx1jjXPUZlp,1,19/7/2025,19/7/2025,10/7/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,2.0,2.0,2 checked out of 2,2.0,1.0,EXISTING/Repeat Attendee,"10, 40",2.0,NaN,17,2025-07-10,0 checked out of 1,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,recqe2XUQ0eSk0CTf,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Events > All,1,NaN,London,4,1.0,100%,17.0,Academy,17.0,No,Yes,She/ Her,Mixed - White and Black African,I will be joining in-person,Yes,Mixed,NaN,Stemette Newsletter,NaN,NaN,NaN,rechqToGPUTdlChai,NaN,NaN,<NA>,12/5/2025 2:37pm,NEW/First-time Attendee,recrPMUI4ZCTOkgop,30/4/2025 9:01pm,NaN,NaN,0,NaN,NaN,30/4/2025,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,1.0,2.0,1 checked out of 2,1.0,1.0,NEW/First-time Attendee,10,1.0,NaN,17,2025-04-30,0 checked out of 1,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,recUdaEr0T20VEL5N,"""Achieve, Connect, Grow: Adventures Season Panel 25""",Adventures,STEAM Seasons 2025,Panel & Speed Networking,2025,2025 - Q3 - [7] July,Ev

In [61]:
Att_v4_pivot_region = pd.pivot_table(Att_v4,
    values=["Record_ID", "Record_ID (Unique)"], # what to count
    index="Region_x", # rows
    columns="Stream", # columns
    aggfunc={"Record_ID":"count", "Record_ID (Unique)":"nunique"}, # aggregation
    fill_value=0,
    margins=True, # adds totals row/col
    margins_name="Total" # label for totals
)

# Flatten the multi-level columns for easier use
Att_v4_pivot_region.columns = [
    f"{val}_{col}" for val, col in Att_v4_pivot_region.columns
]
Att_v4_pivot_region = Att_v4_pivot_region.reset_index()

Att_v4_pivot_region

,Region_x,Record_ID_Academy,Record_ID_Connect,Record_ID_Junior,Record_ID_Over 26,Record_ID_Under 4,Record_ID_Total,Record_ID (Unique)_Academy,Record_ID (Unique)_Connect,Record_ID (Unique)_Junior,Record_ID (Unique)_Over 26,Record_ID (Unique)_Under 4,Record_ID (Unique)_Total
0,East Midlands,18,3,4,0,0,25,16,2,4,0,0,22
1,East of England,42,29,34,0,0,105,38,16,14,0,0,68
2,London,261,180,154,8,1,604,231,101,81,8,1,419
3,North East,12,9,0,1,0,22,12,9,0,1,0,22
4,North West,25,7,5,0,0,37,20,6,5,0,0,31
5,Northern Ireland,1,0,1,0,0,2,1,0,1,0,0,2
6,Outside the UK,11,1,0,0,0,12,10,1,0,0,0,11
7,Postcode Unmatched,0,0,1,0,0,1,0,0,1,0,0,1
8,Region TBC,47,26,34,1,0,108,45,25,34,1,0,105
9,Scotland,10,3,3,0,0,16,8,3,2,0,0,13


In [62]:
Att_v4_pivot_date = pd.pivot_table(
    Att_v3,
    values=["Record_ID", "Record_ID (Unique)"],
    index="Year-Quarter-Month",
    columns="Stream",
    aggfunc={"Record_ID":"count", "Record_ID (Unique)":"nunique"},
    fill_value=0,
    margins=True,
    margins_name="Total"
)

# Flatten the multi-level columns for easier use
Att_v4_pivot_date.columns = [
    f"{val}_{col}" for val, col in Att_v4_pivot_date.columns
]
Att_v4_pivot_date = Att_v4_pivot_date.reset_index()

Att_v4_pivot_date

,Year-Quarter-Month,Record_ID_Academy,Record_ID_Connect,Record_ID_Junior,Record_ID_Over 26,Record_ID_Under 4,Record_ID_Total,Record_ID (Unique)_Academy,Record_ID (Unique)_Connect,Record_ID (Unique)_Junior,Record_ID (Unique)_Over 26,Record_ID (Unique)_Under 4,Record_ID (Unique)_Total
0,2024 - Q1 - [1] January,16,0,0,1,0,17,16,0,0,1,0,17
1,2024 - Q1 - [2] February,24,38,15,0,0,77,24,36,14,0,0,74
2,2024 - Q1 - [3] March,27,14,7,0,0,48,27,14,7,0,0,48
3,2024 - Q2 - [4] April,26,33,34,0,0,93,26,29,32,0,0,87
4,2024 - Q2 - [5] May,64,25,24,1,0,114,63,25,24,1,0,113
5,2024 - Q2 - [6] June,8,4,0,0,0,12,8,4,0,0,0,12
6,2024 - Q3 - [7] July,29,29,33,1,0,92,29,25,26,1,0,81
7,2024 - Q3 - [8] August,85,17,8,2,0,112,80,16,8,2,0,106
8,2024 - Q3 - [9] September,26,0,0,8,0,34,26,0,0,8,0,34
9,2024 - Q4 - [10] October,49,37,27,2,0,115,46,31,25,2,0,103


In [63]:
Att_v4_pivot_subtype = pd.pivot_table(
    Att_v3,
    values=["Record_ID", "Record_ID (Unique)"],
    index="Event Sub-Type",
    columns="Stream",
    aggfunc={"Record_ID":"count", "Record_ID (Unique)":"nunique"},
    fill_value=0,
    margins=True,
    margins_name="Total"
)

# Flatten the multi-level columns for easier use
Att_v4_pivot_subtype.columns = [
    f"{val}_{col}" for val, col in Att_v4_pivot_subtype.columns
]
Att_v4_pivot_subtype = Att_v4_pivot_subtype.reset_index()

Att_v4_pivot_subtype

,Event Sub-Type,Record_ID_Academy,Record_ID_Connect,Record_ID_Junior,Record_ID_Over 26,Record_ID_Under 4,Record_ID_Total,Record_ID (Unique)_Academy,Record_ID (Unique)_Connect,Record_ID (Unique)_Junior,Record_ID (Unique)_Over 26,Record_ID (Unique)_Under 4,Record_ID (Unique)_Total
0,Explore (Half-term programmes),52,116,122,1,0,291,51,91,86,1,0,229
1,Hackathon,142,155,154,4,1,456,131,109,112,4,1,354
2,Mentorship STS,179,1,0,1,0,181,178,1,0,1,0,180
3,Outbox Entrepreneurs,26,0,0,8,0,34,26,0,0,8,0,34
4,Panel & Speed Networking,81,62,1,3,0,147,79,57,1,3,0,140
5,Stemettes Certification Academy (SCA),76,3,0,0,0,79,71,3,0,0,0,74
6,Stemettes Leadership Academy (SLA),46,1,0,2,0,49,44,1,0,2,0,47
7,Total,602,338,277,19,1,1237,532,214,167,19,1,929


In [64]:
Att_v4_pivot_steamseason = pd.pivot_table(
    Att_v3[Att_v3["STEAM Season"] != "Not Applicable"],
    values=["Record_ID", "Record_ID (Unique)"],
    index="STEAM Season",
    columns="Stream",
    aggfunc={"Record_ID":"count", "Record_ID (Unique)":"nunique"},
    fill_value=0,
    margins=True,
    margins_name="Total"
)

Att_v4_pivot_steamseason.columns = [
    f"{val}_{col}" for val, col in Att_v4_pivot_steamseason.columns
]
Att_v4_pivot_steamseason = Att_v4_pivot_steamseason.reset_index()
Att_v4_pivot_steamseason

,STEAM Season,Record_ID_Academy,Record_ID_Connect,Record_ID_Junior,Record_ID_Over 26,Record_ID_Under 4,Record_ID_Total,Record_ID (Unique)_Academy,Record_ID (Unique)_Connect,Record_ID (Unique)_Junior,Record_ID (Unique)_Over 26,Record_ID (Unique)_Under 4,Record_ID (Unique)_Total
0,Adventures,107,61,59,3,0,230,97,53,54,3,0,207
1,Ambitions,71,97,77,0,1,246,71,78,62,0,1,212
2,Insights,86,89,60,1,0,236,82,77,46,1,0,206
3,Visions,56,48,27,3,0,134,52,42,25,3,0,121
4,Total,320,295,223,7,1,846,283,192,140,7,1,620


In [65]:
# 1. Build the pivot
Att_v4_pivot_steamseason = pd.pivot_table(
    Att_v3[Att_v3["STEAM Season"] != "Not Applicable"],
    values=["Record_ID", "Record_ID (Unique)"],
    index="STEAM Season",
    columns="Stream",
    aggfunc={"Record_ID": "count", "Record_ID (Unique)": "nunique"},
    fill_value=0,
    margins=True,
    margins_name="Total"
)

# 2. Flatten MultiIndex columns safely
Att_v4_pivot_steamseason.columns = [
    "_".join([str(x) for x in col if x not in ("", None)])
    if isinstance(col, tuple) else str(col)
    for col in Att_v4_pivot_steamseason.columns
]

# 3. Bring STEAM Season back as a column
Att_v4_pivot_steamseason = Att_v4_pivot_steamseason.reset_index()

Att_v4_pivot_steamseason

,STEAM Season,Record_ID_Academy,Record_ID_Connect,Record_ID_Junior,Record_ID_Over 26,Record_ID_Under 4,Record_ID_Total,Record_ID (Unique)_Academy,Record_ID (Unique)_Connect,Record_ID (Unique)_Junior,Record_ID (Unique)_Over 26,Record_ID (Unique)_Under 4,Record_ID (Unique)_Total
0,Adventures,107,61,59,3,0,230,97,53,54,3,0,207
1,Ambitions,71,97,77,0,1,246,71,78,62,0,1,212
2,Insights,86,89,60,1,0,236,82,77,46,1,0,206
3,Visions,56,48,27,3,0,134,52,42,25,3,0,121
4,Total,320,295,223,7,1,846,283,192,140,7,1,620


## Export cleaned datasets for downstream projects

This cell saves the cleaned/enriched datasets produced in **Project 1** so they can be loaded directly in **Project 2** (visual analytics) and any later work without re-running the full ETL every time.

In [ ]:
from pathlib import Path
import pickle

out_dir = Path("outputs")
out_dir.mkdir(exist_ok=True)

def save_pickle(obj, name):
    path = out_dir / f"{name}.pkl"
    with open(path, "wb") as f:
        pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
    print(f"Saved: {path.resolve()}")

# Core versions
for name in ["Att_v1","Att_v2","Att_v3","Att_v4"]:
    if name in globals():
        save_pickle(globals()[name], name)

# Useful analysis-ready artefacts (only saved if they exist)
for name in ["Att_v4_pivot_region","Att_v4_pivot_date","Att_v4_pivot_subtype","Att_v4_pivot_steamseason","Fb_new"]:
    if name in globals():
        save_pickle(globals()[name], name)

print("Done. You can now open Project 2 and it will auto-load these files from /outputs.")
